# Basic settings and Importing required packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install nilearn
!pip install einops
!pip install beartype
!pip install numpy==1.23.0
import sys
sys.path.append('/content/drive/MyDrive/GraduationStudy/HOT')

In [3]:
import os
import torch
import math
import numpy as np
import pandas as pd
import torch.nn as nn
import networkx as nx
from einops import *
from tqdm import tqdm
from google.colab import drive
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import MultiheadAttention
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
import torch.utils.checkpoint
import zipfile
import gzip
import shutil


from models.MemoryModel import MemoryModel, compute_src_dst_node_time_shifts
from models.modules import TimeEncoder
from utils.utils import NeighborSampler
from utils.utils import get_neighbor_sampler, NegativeEdgeSampler
from utils.DataLoader import get_idx_data_loader
from models.block_recurrent_transformer_pytorch import BlockRecurrentTransformer
from nilearn import image, maskers, datasets

import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
%cd drive
%cd MyDrive
%cd GraduationStudy


/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/GraduationStudy


# Saving data & Data preprocessing

The helper functions are defined in the cell below.

In [ ]:
# Normalization of the fMRI timeseries
# Shape of timeseries : (# of timestamps, # of ROIs)
def normalize(ts):
  return (ts-torch.mean(ts, dim=0, keepdims=True))/(torch.std(ts, dim=0, keepdims=True)+1e-10)

# corrcoef based on
# https://github.com/pytorch/pytorch/issues/1254
def corrcoef(x):
    mean_x = torch.mean(x, 1, keepdim=True)
    xm = x.sub(mean_x.expand_as(x))
    c = xm.mm(xm.t())
    c = c / (x.size(1) - 1)
    d = torch.diag(c)
    stddev = torch.pow(d, 0.5)
    c = c.div(stddev.expand_as(c))
    c = c.div(stddev.expand_as(c).t())
    c = torch.clamp(c, -1.0, 1.0)
    return c

# process_dynamic_fc
# Source : https://github.com/egyptdj/stagin/blob/main/util/bold.py
def get_fc(timeseries, sampling_point, window_size, self_loop):
    fc = corrcoef(timeseries[sampling_point:sampling_point+window_size].T)
    if not self_loop: fc-= torch.eye(fc.shape[0])
    return fc

def get_minibatch_fc(minibatch_timeseries, sampling_point, window_size, self_loop):
    fc_list = []
    for timeseries in minibatch_timeseries:
        fc = get_fc(timeseries, sampling_point, window_size, self_loop)
        fc_list.append(fc)
    return torch.stack(fc_list)


def process_dynamic_fc(minibatch_timeseries, window_size, window_stride, dynamic_length=None, sampling_init=None, self_loop=True):
    # assumes input shape [minibatch x time x node]
    # output shape [minibatch x time x node x node]
    if dynamic_length is None:
        dynamic_length = minibatch_timeseries.shape[1]
        sampling_init = 0
    else:
        if isinstance(sampling_init, int):
            assert minibatch_timeseries.shape[1] > sampling_init + dynamic_length
    assert sampling_init is None or isinstance(sampling_init, int)
    assert minibatch_timeseries.ndim==3
    assert dynamic_length > window_size

    if sampling_init is None:
        sampling_init = randrange(minibatch_timeseries.shape[1]-dynamic_length+1)
    sampling_points = list(range(sampling_init, sampling_init+dynamic_length-window_size, window_stride))

    minibatch_fc_list = [get_minibatch_fc(minibatch_timeseries, sampling_point, window_size, self_loop) for sampling_point in sampling_points]
    dynamic_fc = torch.stack(minibatch_fc_list, dim=1)

    return dynamic_fc, sampling_points

def readtxt(filePath):
  # Read the .txt file
  data = []
  with open(filePath, 'r') as file:
      for line in file:
          # Split the line by whitespace or commas and convert to float
          row = [float(value) for value in line.split()]
          data.append(row)

  # Convert list to a PyTorch tensor
  tensor = torch.tensor(data)

  return tensor



The fMRI data should be downloaded from ConnectomeDB website.
(https://db.humanconnectome.org/app/template/Login.vm)

The directory tree should be constructed like the following diagram.

MyDrive\
$\quad$├─GraduationStudy\
$\quad$$\quad$$\quad$      ├─Emotion Task Original Files\
$\quad$$\quad$$\quad$$\quad$$\quad$            ├─######_3T_tfMRI_EMOTION_preproc.zip\
$\quad$$\quad$$\quad$$\quad$$\quad$            ├─@@@@@@_3T_tfMRI_EMOTION_preproc.zip\
$\quad$$\quad$$\quad$$\quad$$\quad$            ├─......\
$\quad$$\quad$$\quad$$\quad$$\quad$            ├─&&&&&&_3T_tfMRI_EMOTION_preproc.zip\
$\quad$$\quad$$\quad$├─DatasetSplit


The cell below constructs two 4D tensors of shape (100, 15, 116, 116)\
"tripletTensor40.pth" holds the 40th Quantile Dataset.\
"tripletTensor80.pth" holds the 80th Quantile Dataset.

In [ ]:
baseDir='Emotion Task Original Files'
prefix='MNINonLinear/Results/tfMRI_EMOTION_LR'
subjectIDs=[]
for zipFile in os.listdir(baseDir):
  if zipFile[-1]!='p':
    continue
  subjectID=zipFile[0:6]
  print(subjectID)
  subjectIDs.append(subjectID)
  files_to_extract = [os.path.join(subjectID, prefix,'EVs/fear.txt'), os.path.join(subjectID, prefix, 'tfMRI_EMOTION_LR.nii.gz')]
  with zipfile.ZipFile(os.path.join(baseDir,zipFile), 'r') as zip_ref:
    for file in files_to_extract:
        zip_ref.extract(file, path=os.path.join(baseDir, subjectID))

for ID in subjectIDs:
  gzFile = os.path.join(baseDir, ID, ID, prefix,'tfMRI_EMOTION_LR.nii.gz')
  niiFile = os.path.join(baseDir, ID, 'tfMRI_EMOTION_LR.nii')
  txtFile = os.path.join(baseDir, ID, ID, prefix, 'EVs/fear.txt')
  with gzip.open(gzFile, 'rb') as f_in:
    with open(niiFile, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
  shutil.copyfile(txtFile, os.path.join(baseDir,ID,'fear.txt'))

atlasData=datasets.fetch_atlas_aal(data_dir=baseDir)
for ID in subjectIDs:
  img=image.load_img(os.path.join(baseDir, ID, 'tfMRI_EMOTION_LR.nii'))
  masker = maskers.NiftiLabelsMasker(image.load_img(atlasData['maps']))
  timeseries = masker.fit_transform(img) # (numTP, N=116)
  torch.save(timeseries, os.path.join(baseDir, ID, 'Emotion_Task.pth'))

windowSize=20
triplet40=[]
triplet80=[]
for ID in subjectIDs:
  timeSeries=torch.from_numpy(torch.load(os.path.join(baseDir, ID, 'Emotion_Task.pth')))
  timeSeries=normalize(timeSeries)
  print(timeSeries.shape)
  onsetFile=os.path.join(baseDir, ID, 'fear.txt')
  onsetTensor=readtxt(onsetFile)
  onsetStamp1=math.floor(onsetTensor[0,0]/0.72)
  onsetStamp2=math.floor(onsetTensor[1,0]/0.72)
  onsetStamp3=math.floor(onsetTensor[2,0]/0.72)

  fcList=[]
  onsets=[onsetStamp1, onsetStamp2, onsetStamp3]
  for onset in onsets:
    for stride in range(-3, 12, 3):
      fc=get_fc(timeSeries, onset+stride, windowSize, True)
      fcList.append(fc)
  fcSubject=torch.stack(fcList)

  samplePoints=[]
  for onset in onsets:
    for stride in range(-3, 12, 3):
      samplePoints.append(onset+stride)

  triplet40S=[]
  triplet80S=[]
  for sampleidx, fc2D in enumerate(fcSubject):
    triplet80List=[]
    fc80=fc2D[40].clone()
    fcSort80=fc80.sort(descending=True)
    fc80[fcSort80.indices[math.ceil(116*0.8):]]=0
    idx80=torch.nonzero(fc80)[:,0]
    idx1D=samplePoints[sampleidx]*torch.ones_like(idx80)
    for i in range(idx80.shape[0]):
      triplet80LIst.append([40, idx80[i].item(), idx1D[i].item()])

    fcTemp=fc2D.flatten()
    fcSort=fcTemp.sort(descending=True)
    fcTemp[fcSort.indices[math.ceil(116*116*0.4):]]=0
    fcMasked=fcTemp.reshape(fc2D.shape)
    idx2D, idx3D=torch.nonzero(fcMasked, as_tuple=True)
    idx1D=samplePoints[sampleidx]*torch.ones_like(idx2D)
    assert idx1D.shape[0]==math.ceil(116*116*0.4)
    for j in range(idx1D.shape[0]):
      triplet40S.append([idx2D[j].item(), idx3D[j].item(), idx1D[j].item()])
    for j in range(idx1D.shape[0]):
      if idx2D[j]!=40:
        triplet80List.append([idx2D[j].item(), idx3D[j].item(), idx1D[j].item()])

    if len(triplet80S)>math.ceil(116*116*0.4):
      subVal=len(triplet80S)-math.ceil(116*116*0.4)
      for k in range(subVal):
        valRemove=random.randint(0, len(triplet80S)-1)
        triplet80S.pop(valRemove)
      while not (torch.tensor(triplet80S)[:,0].unique().shape[0]==116 and torch.tensor(triplet80S)[:,1].unique().shape[0]==116):
        if torch.tensor(triplet80S)[:,0].unique().shape[0]!=116:
          triplet80S.pop(random.randint(0, len(triplet80S)-1))
          missing=[i for i in range(116) if i not in (torch.tensor(triplet80S))[:,0].unique()]
          for k in range(len(missing)):
            triplet80S.append([missing[k], random.randint(0, 115), samplePoints[sampleidx]])
        if torch.tensor(triplet80S)[:,0].unique().shape[0]==116 and torch.tensor(triplet80S)[:,1].unique().shape[0]==116:
          break
        if torch.tensor(triplet80S)[:,1].unique().shape[0]!=116:
          triplet80S.pop(random.randint(0, len(triplet80S)-1))
          missing=[i for i in range(116) if i not in (torch.tensor(triplet80S))[:,1].unique()]
          for k in range(len(missing)):
            triplet80S.append([random.randint(0, 115), missing[k], samplePoints[sampleidx]])
    elif len(triplet80S)<math.ceil(116*116*0.4):
      addVal=math.ceil(116*116*0.4)-len(triplet80S)
      for k in range(addVal):
        val1=random.randint(0, 115)
        while val1==40:
          val1=random.randint(0, 115)
        valZero=torch.nonzero(fc2D[fc2D==val1])[:,0]
        cand=[i for i in range(116) if i not in valZero]
        val2=random.choice(cand)
        triplet80S.append([val1, val2, samplePoints[sampleidx]])

    triplet403D=torch.tensor(triplet40S).to('cuda')
    triplet40S.append(triplet403D)

    assert len(triplet80S)==math.ceil(116*116*0.4)
    triplet803D=torch.tensor(triplet80S).to('cuda')
    assert torch.unique(triplet803D[:,0]).shape[0]==116
    assert torch.unique(triplet803D[:,1]).shape[0]==116

    triplet80S.append(triplet803D)

  triplet40S=torch.stack(triplet40S).to('cuda')
  triplet40.append(triplet40S)

  triplet80S=torch.tensor(triplet80S).to('cuda')
  triplet80.append(triplet80S)

tripletTensor=torch.stack(triplet40).to('cuda')
torch.save(tripletTensor, os.path.join('DatasetSplit','tripletTensor40.pth'))
triplet80Tensor=torch.stack(triplet80).to('cuda')
torch.save(triplet80Tensor, os.path.join('DatasetSplit','tripletTensor80.pth'))

The cell below divides the whole dataset tensor into training, validation, and test data.\
"train@.pth", "valid@.pth", and "test@.pth" holds the 40th Quantile  train, validation, and test data of @-th subject, respectively.\
"train80@.pth", "valid80@.pth", and "test80@.pth" holds the 80th Quantile  train, validation, and test data of @-th subject, respectively.

In [ ]:
numList=[i for i in range(100)]
random.shuffle(numList)
trainIdx=numList[0:16]
validIdx=numList[16:24]
testIdx=numList[24:34]


triplet40s=torch.load(os.path.join('DatasetSplit','tripletTensor40.pth')).to('cuda')
triplet80s=torch.load(os.path.join('DatasetSplit','tripletTensor80.pth')).to('cuda')
trainTriplet=triplet40s[trainIdx]
testTriplet=triplet40s[testIdx]
validTriplet=triplet40s[validIdx]
train80=triplet80s[trainIdx]
test80=triplet80s[testIdx]
valid80=triplet80s[validIdx]
for i in range(16):
  torch.save(trainTriplet[i].squeeze(0), os.path.join('DatasetSplit','train'+str(i)+'.pth'))
  torch.save(train80[i].squeeze(0), os.path.join('DatasetSplit','train80'+str(i)+'.pth'))
for i in range(8):
  torch.save(validTriplet[i].squeeze(0), os.path.join('DatasetSplit','valid'+str(i)+'.pth'))
  torch.save(valid80[i].squeeze(0), os.path.join('DatasetSplit','valid80'+str(i)+'.pth'))
for i in range(10):
  torch.save(testTriplet[i].squeeze(0), os.path.join('DatasetSplit','test'+str(i)+'.pth'))
  torch.save(test80[i].squeeze(0), os.path.join('DatasetSplit','test80'+str(i)+'.pth'))

<ipython-input-12-1223466b73ce>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  triplet4D=torch.load(os.path.join('DatasetSplit','tripletTensor.pth')).to('cuda')
<ipython-i

# Models - HOT and Triadic Decoder

The cell below is a definition of the HOT encoder, introduced in [Besta et al., 2024](https://arxiv.org/abs/2311.18526).

In [ ]:
class HOT(nn.Module):

    def __init__(self, node_raw_features: np.ndarray, edge_raw_features: np.ndarray, neighbor_sampler: NeighborSampler,
                 time_feat_dim: int, channel_embedding_dim: int, patch_size: int = 1, num_layers: int = 2, num_heads: int = 4,
                 block_size: int = 16, num_state_vectors: int = 32, segment_size: int = 32, num2hop: int = 0,
                 dropout: float = 0.1, max_input_sequence_length: int = 65536, device: str = 'cpu'):
        """
        DyGFormer model.
        :param node_raw_features: ndarray, shape (num_nodes + 1, node_feat_dim)
        :param edge_raw_features: ndarray, shape (num_edges + 1, edge_feat_dim)
        :param neighbor_sampler: neighbor sampler
        :param time_feat_dim: int, dimension of time features (encodings)
        :param channel_embedding_dim: int, dimension of each channel embedding
        :param patch_size: int, patch size

        :param num_layers: int, number of transformer layers
        :param num_heads: int, number of attention heads
        :param dropout: float, dropout rate
        :param max_input_sequence_length: int, maximal length of the input sequence for each node
        :param device: str, device
        """
        super(HOT, self).__init__()

        self.node_raw_features = torch.from_numpy(node_raw_features.astype(np.float32)).to(device)
        self.edge_raw_features = torch.from_numpy(edge_raw_features.astype(np.float32)).to(device)

        self.neighbor_sampler = neighbor_sampler
        self.node_feat_dim = self.node_raw_features.shape[1] + 2
        self.edge_feat_dim = self.edge_raw_features.shape[1]
        self.time_feat_dim = time_feat_dim
        self.channel_embedding_dim = channel_embedding_dim
        self.patch_size = patch_size
        self.num_layers = num_layers
        self.block_size = block_size
        self.segment_size = segment_size
        self.num_state_vectors = num_state_vectors
        self.num_heads = num_heads
        self.dropout = dropout
        self.max_input_sequence_length = max_input_sequence_length
        self.device = device

        self.num2hop = num2hop

        self.time_encoder = TimeEncoder(time_dim=time_feat_dim).to(self.device)

        self.neighbor_co_occurrence_feat_dim = self.channel_embedding_dim
        self.neighbor_co_occurrence_encoder = NeighborCooccurrenceEncoder(neighbor_co_occurrence_feat_dim=self.neighbor_co_occurrence_feat_dim, device=self.device)

        self.projection_layer = nn.ModuleDict({
            'node': nn.Linear(in_features=self.patch_size * self.node_feat_dim, out_features=self.channel_embedding_dim, bias=True),
            'edge': nn.Linear(in_features=self.patch_size * self.edge_feat_dim, out_features=self.channel_embedding_dim, bias=True),
            'time': nn.Linear(in_features=self.patch_size * self.time_feat_dim, out_features=self.channel_embedding_dim, bias=True),
            'neighbor_co_occurrence': nn.Linear(in_features=self.patch_size * self.neighbor_co_occurrence_feat_dim, out_features=self.channel_embedding_dim, bias=True)
        }).to(self.device)

        self.num_channels = 4

        self.brt = BlockRecurrentTransformer(
            dim = 2 * self.num_channels * self.channel_embedding_dim,
            depth = self.num_layers,
            dim_head = self.num_channels * self.channel_embedding_dim // (self.num_heads // 2),
            heads = self.num_heads,
            max_seq_len = self.segment_size,
            block_width = self.block_size,
            num_state_vectors = self.num_state_vectors,
            recurrent_layers = (1, ),
            use_compressed_mem = False,
            use_flash_attn = False
        ).to(self.device)

        self.output_layer = nn.Linear(in_features=self.num_channels * self.channel_embedding_dim, out_features=self.node_raw_features.shape[1], bias=True).to(self.device)

    def compute_src_dst_node_temporal_embeddings(self, src_node_ids: np.ndarray, dst_node_ids: np.ndarray, node_interact_times: np.ndarray):
        """
        compute source and destination node temporal embeddings
        :param src_node_ids: ndarray, shape (batch_size, )
        :param dst_node_ids: ndarray, shape (batch_size, )
        :param node_interact_times: ndarray, shape (batch_size, )
        :return:
        """

        # Extract 1-hop and 2-hop interactions
        src_firt_hop_lengths, src_nodes_neighbor_ids_list, src_nodes_edge_ids_list, src_nodes_neighbor_times_list = \
            self.neighbor_sampler.get_all_second_hop_neighbors(node_ids=src_node_ids, node_interact_times=node_interact_times,
                                                                max_seq_len=self.max_input_sequence_length, max_2hop=self.num2hop)

        dst_firt_hop_lengths, dst_nodes_neighbor_ids_list, dst_nodes_edge_ids_list, dst_nodes_neighbor_times_list = \
            self.neighbor_sampler.get_all_second_hop_neighbors(node_ids=dst_node_ids, node_interact_times=node_interact_times,
                                                                max_seq_len=self.max_input_sequence_length, max_2hop=self.num2hop)

        # pad the sequences
        src_padded_nodes_neighbor_ids, src_padded_nodes_edge_ids, src_padded_nodes_neighbor_times = \
            self.pad_sequences(node_ids=src_node_ids, node_interact_times=node_interact_times, nodes_neighbor_ids_list=src_nodes_neighbor_ids_list,
                               nodes_edge_ids_list=src_nodes_edge_ids_list, nodes_neighbor_times_list=src_nodes_neighbor_times_list,
                               patch_size=self.patch_size, max_input_sequence_length=1048576)

        dst_padded_nodes_neighbor_ids, dst_padded_nodes_edge_ids, dst_padded_nodes_neighbor_times = \
            self.pad_sequences(node_ids=dst_node_ids, node_interact_times=node_interact_times, nodes_neighbor_ids_list=dst_nodes_neighbor_ids_list,
                               nodes_edge_ids_list=dst_nodes_edge_ids_list, nodes_neighbor_times_list=dst_nodes_neighbor_times_list,
                               patch_size=self.patch_size, max_input_sequence_length=1048576)

        # get HO encoding
        src_padded_nodes_neighbor_co_occurrence_features, dst_padded_nodes_neighbor_co_occurrence_features = \
            self.neighbor_co_occurrence_encoder(src_padded_nodes_neighbor_ids=src_padded_nodes_neighbor_ids,
                                                dst_padded_nodes_neighbor_ids=dst_padded_nodes_neighbor_ids)

        # get features
        src_padded_nodes_neighbor_node_raw_features, src_padded_nodes_edge_raw_features, src_padded_nodes_neighbor_time_features = \
            self.get_features(node_interact_times=node_interact_times, padded_nodes_neighbor_ids=src_padded_nodes_neighbor_ids,
                              padded_nodes_edge_ids=src_padded_nodes_edge_ids, padded_nodes_neighbor_times=src_padded_nodes_neighbor_times, time_encoder=self.time_encoder)

        dst_padded_nodes_neighbor_node_raw_features, dst_padded_nodes_edge_raw_features, dst_padded_nodes_neighbor_time_features = \
            self.get_features(node_interact_times=node_interact_times, padded_nodes_neighbor_ids=dst_padded_nodes_neighbor_ids,
                              padded_nodes_edge_ids=dst_padded_nodes_edge_ids, padded_nodes_neighbor_times=dst_padded_nodes_neighbor_times, time_encoder=self.time_encoder)


        # Add one-hot encoding
        batch_size = len(src_padded_nodes_neighbor_node_raw_features)

        new_src_padded_nodes_neighbor_node_raw_features = torch.empty((src_padded_nodes_neighbor_node_raw_features.shape[0],
                                                                       src_padded_nodes_neighbor_node_raw_features.shape[1],
                                                                       src_padded_nodes_neighbor_node_raw_features.shape[2] + 2)).to(self.device)

        new_dst_padded_nodes_neighbor_node_raw_features = torch.empty((dst_padded_nodes_neighbor_node_raw_features.shape[0],
                                                                       dst_padded_nodes_neighbor_node_raw_features.shape[1],
                                                                       dst_padded_nodes_neighbor_node_raw_features.shape[2] + 2)).to(self.device)

        for i in range(batch_size):
            if (src_firt_hop_lengths[i] != 0):
                new_src_padded_nodes_neighbor_node_raw_features[i, -src_firt_hop_lengths[i]:, :] = \
                    torch.cat([src_padded_nodes_neighbor_node_raw_features[i, -src_firt_hop_lengths[i]:, :],
                            torch.ones((src_firt_hop_lengths[i], 1)).float().to(self.device),
                            torch.zeros((src_firt_hop_lengths[i], 1)).float().to(self.device),], dim=1)
                new_src_padded_nodes_neighbor_node_raw_features[i, :-src_firt_hop_lengths[i], :] = \
                    torch.cat([src_padded_nodes_neighbor_node_raw_features[i, :-src_firt_hop_lengths[i], :],
                            torch.zeros((src_padded_nodes_neighbor_node_raw_features.shape[1] - src_firt_hop_lengths[i], 1)).float().to(self.device),
                            torch.ones((src_padded_nodes_neighbor_node_raw_features.shape[1] - src_firt_hop_lengths[i], 1)).float().to(self.device)], dim=1)

            else:
                new_src_padded_nodes_neighbor_node_raw_features[i, :, :] = \
                    torch.cat([src_padded_nodes_neighbor_node_raw_features[i, :, :],
                            torch.zeros((src_padded_nodes_neighbor_node_raw_features.shape[1], 1)).float().to(self.device),
                            torch.ones((src_padded_nodes_neighbor_node_raw_features.shape[1], 1)).float().to(self.device)], dim=1)

            if (dst_firt_hop_lengths[i] != 0):
                new_dst_padded_nodes_neighbor_node_raw_features[i, -dst_firt_hop_lengths[i]:, :] = \
                    torch.cat([dst_padded_nodes_neighbor_node_raw_features[i, -dst_firt_hop_lengths[i]:, :],
                            torch.ones((dst_firt_hop_lengths[i], 1)).float().to(self.device),
                            torch.zeros((dst_firt_hop_lengths[i], 1)).float().to(self.device)], dim=1)
                new_dst_padded_nodes_neighbor_node_raw_features[i, :-dst_firt_hop_lengths[i], :] = \
                    torch.cat([dst_padded_nodes_neighbor_node_raw_features[i, :-dst_firt_hop_lengths[i], :],
                            torch.zeros((dst_padded_nodes_neighbor_node_raw_features.shape[1] - dst_firt_hop_lengths[i], 1)).float().to(self.device),
                            torch.ones((dst_padded_nodes_neighbor_node_raw_features.shape[1] - dst_firt_hop_lengths[i], 1)).float().to(self.device)], dim=1)
            else:
                new_dst_padded_nodes_neighbor_node_raw_features[i, :, :] = \
                    torch.cat([dst_padded_nodes_neighbor_node_raw_features[i, :, :],
                            torch.zeros((dst_padded_nodes_neighbor_node_raw_features.shape[1], 1)).float().to(self.device),
                            torch.ones((dst_padded_nodes_neighbor_node_raw_features.shape[1], 1)).float().to(self.device)], dim=1)


        # Patching
        src_patches_nodes_neighbor_node_raw_features, src_patches_nodes_edge_raw_features, \
        src_patches_nodes_neighbor_time_features, src_patches_nodes_neighbor_co_occurrence_features = \
            self.get_patches(padded_nodes_neighbor_node_raw_features=new_src_padded_nodes_neighbor_node_raw_features,
                             padded_nodes_edge_raw_features=src_padded_nodes_edge_raw_features,
                             padded_nodes_neighbor_time_features=src_padded_nodes_neighbor_time_features,
                             padded_nodes_neighbor_co_occurrence_features=src_padded_nodes_neighbor_co_occurrence_features,
                             patch_size=self.patch_size)

        dst_patches_nodes_neighbor_node_raw_features, dst_patches_nodes_edge_raw_features, \
        dst_patches_nodes_neighbor_time_features, dst_patches_nodes_neighbor_co_occurrence_features = \
            self.get_patches(padded_nodes_neighbor_node_raw_features=new_dst_padded_nodes_neighbor_node_raw_features,
                             padded_nodes_edge_raw_features=dst_padded_nodes_edge_raw_features,
                             padded_nodes_neighbor_time_features=dst_padded_nodes_neighbor_time_features,
                             padded_nodes_neighbor_co_occurrence_features=dst_padded_nodes_neighbor_co_occurrence_features,
                             patch_size=self.patch_size)

        # Alignment
        src_patches_nodes_neighbor_node_raw_features = self.projection_layer['node'](src_patches_nodes_neighbor_node_raw_features)
        src_patches_nodes_edge_raw_features = self.projection_layer['edge'](src_patches_nodes_edge_raw_features)
        src_patches_nodes_neighbor_time_features = self.projection_layer['time'](src_patches_nodes_neighbor_time_features)
        src_patches_nodes_neighbor_co_occurrence_features = self.projection_layer['neighbor_co_occurrence'](src_patches_nodes_neighbor_co_occurrence_features)

        dst_patches_nodes_neighbor_node_raw_features = self.projection_layer['node'](dst_patches_nodes_neighbor_node_raw_features)
        dst_patches_nodes_edge_raw_features = self.projection_layer['edge'](dst_patches_nodes_edge_raw_features)
        dst_patches_nodes_neighbor_time_features = self.projection_layer['time'](dst_patches_nodes_neighbor_time_features)
        dst_patches_nodes_neighbor_co_occurrence_features = self.projection_layer['neighbor_co_occurrence'](dst_patches_nodes_neighbor_co_occurrence_features)


        # Concatenation
        src_num_patches = src_patches_nodes_neighbor_node_raw_features.shape[1]
        dst_num_patches = dst_patches_nodes_neighbor_node_raw_features.shape[1]

        big_patch_size = src_num_patches

        if src_num_patches > dst_num_patches:
            dst_patches_nodes_neighbor_node_raw_features = F.pad(dst_patches_nodes_neighbor_node_raw_features, (0, 0, src_num_patches - dst_num_patches, 0), value=0)
            dst_patches_nodes_edge_raw_features = F.pad(dst_patches_nodes_edge_raw_features, (0, 0, src_num_patches - dst_num_patches, 0), value=0)
            dst_patches_nodes_neighbor_time_features = F.pad(dst_patches_nodes_neighbor_time_features, (0, 0, src_num_patches - dst_num_patches, 0), value=0)
            dst_patches_nodes_neighbor_co_occurrence_features = F.pad(dst_patches_nodes_neighbor_co_occurrence_features, (0, 0, src_num_patches - dst_num_patches, 0), value=0)
        elif src_num_patches < dst_num_patches:
            big_patch_size = dst_num_patches
            src_patches_nodes_neighbor_node_raw_features = F.pad(src_patches_nodes_neighbor_node_raw_features, (0, 0, dst_num_patches - src_num_patches, 0), value=0)
            src_patches_nodes_edge_raw_features = F.pad(src_patches_nodes_edge_raw_features, (0, 0, dst_num_patches - src_num_patches, 0), value=0)
            src_patches_nodes_neighbor_time_features = F.pad(src_patches_nodes_neighbor_time_features, (0, 0, dst_num_patches - src_num_patches, 0), value=0)
            src_patches_nodes_neighbor_co_occurrence_features = F.pad(src_patches_nodes_neighbor_co_occurrence_features, (0, 0, dst_num_patches - src_num_patches, 0), value=0)

        n_segments = big_patch_size // self.segment_size \
            + (1 if big_patch_size % self.segment_size != 0 else 0)
        if (big_patch_size % self.segment_size == 0):
            first_segment_size = self.segment_size
        else:
            first_segment_size = big_patch_size % self.segment_size

        if (n_segments == 1):
            last_segment_size = big_patch_size
        else:
            last_segment_size = self.segment_size

        src_out = torch.empty(batch_size, big_patch_size, self.num_channels * self.channel_embedding_dim,
                              dtype=src_patches_nodes_edge_raw_features.dtype, device=self.device)
        dst_out = torch.empty(batch_size, big_patch_size, self.num_channels * self.channel_embedding_dim,
                              dtype=src_patches_nodes_edge_raw_features.dtype, device=self.device)
        for i in range(0, n_segments):

            if (i == 0):
                start_segment = 0
                end_segment = start_segment + first_segment_size
            else:
                start_segment = (i - 1) * self.segment_size + first_segment_size
                end_segment = start_segment + self.segment_size

            patches_data = [src_patches_nodes_neighbor_node_raw_features[:, start_segment:end_segment, :],
                            src_patches_nodes_edge_raw_features[:, start_segment:end_segment, :],
                            src_patches_nodes_neighbor_time_features[:, start_segment:end_segment, :],
                            src_patches_nodes_neighbor_co_occurrence_features[:, start_segment:end_segment, :],
                            dst_patches_nodes_neighbor_node_raw_features[:, start_segment:end_segment, :],
                            dst_patches_nodes_edge_raw_features[:, start_segment:end_segment, :],
                            dst_patches_nodes_neighbor_time_features[:, start_segment:end_segment, :],
                            dst_patches_nodes_neighbor_co_occurrence_features[:, start_segment:end_segment, :]]

            # Tensor, shape (batch_size, src_num_patches + dst_num_patches, num_channels, channel_embedding_dim)
            patches_data = torch.stack(patches_data, dim=2)
            # Tensor, shape (batch_size, src_num_patches + dst_num_patches, num_channels * channel_embedding_dim)
            patches_data = patches_data.reshape(batch_size, (end_segment - start_segment), 2 * self.num_channels * self.channel_embedding_dim)

            # Tensor, shape (batch_size, src_num_patches + dst_num_patches, num_channels * channel_embedding_dim)
            if (i > 0 and mem is not None):
                patches_data, mem, stat = self.brt(patches_data, xl_memories=mem, states=stat)
            else:
                patches_data, mem, stat = self.brt(patches_data)


            # src_patches_data, Tensor, shape (batch_size, src_num_patches, num_channels * channel_embedding_dim)
            src_out[:, start_segment:end_segment, :] = patches_data[:, :, :self.num_channels * self.channel_embedding_dim]
            # dst_patches_data, Tensor, shape (batch_size, dst_num_patches, num_channels * channel_embedding_dim)
            dst_out[:, start_segment:end_segment, :] = patches_data[:, :, self.num_channels * self.channel_embedding_dim:]


        # src_patches_data, Tensor, shape (batch_size, num_channels * channel_embedding_dim)
        src_out = torch.mean(src_out, dim=1)
        # dst_patches_data, Tensor, shape (batch_size, num_channels * channel_embedding_dim)
        dst_out = torch.mean(dst_out, dim=1)

        # Tensor, shape (batch_size, node_feat_dim)
        src_node_embeddings = self.output_layer(src_out)
        # Tensor, shape (batch_size, node_feat_dim)
        dst_node_embeddings = self.output_layer(dst_out)

        return src_node_embeddings, dst_node_embeddings

    def pad_sequences(self, node_ids: np.ndarray, node_interact_times: np.ndarray, nodes_neighbor_ids_list: list, nodes_edge_ids_list: list,
                      nodes_neighbor_times_list: list, patch_size: int = 1, max_input_sequence_length: int = 256):
        """
        pad the sequences for nodes in node_ids
        :param node_ids: ndarray, shape (batch_size, )
        :param node_interact_times: ndarray, shape (batch_size, )
        :param nodes_neighbor_ids_list: list of ndarrays, each ndarray contains neighbor ids for nodes in node_ids
        :param nodes_edge_ids_list: list of ndarrays, each ndarray contains edge ids for nodes in node_ids
        :param nodes_neighbor_times_list: list of ndarrays, each ndarray contains neighbor interaction timestamp for nodes in node_ids
        :param patch_size: int, patch size
        :param max_input_sequence_length: int, maximal number of neighbors for each node
        :return:
        """
        assert max_input_sequence_length - 1 > 0, 'Maximal number of neighbors for each node should be greater than 1!'
        max_seq_length = 0
        # first cut the sequence of nodes whose number of neighbors is more than max_input_sequence_length - 1 (we need to include the target node in the sequence)
        for idx in range(len(nodes_neighbor_ids_list)):
            assert len(nodes_neighbor_ids_list[idx]) == len(nodes_edge_ids_list[idx]) == len(nodes_neighbor_times_list[idx])
            if len(nodes_neighbor_ids_list[idx]) > max_input_sequence_length - 1:
                # cut the sequence by taking the most recent max_input_sequence_length interactions
                nodes_neighbor_ids_list[idx] = nodes_neighbor_ids_list[idx][-(max_input_sequence_length - 1):]
                nodes_edge_ids_list[idx] = nodes_edge_ids_list[idx][-(max_input_sequence_length - 1):]
                nodes_neighbor_times_list[idx] = nodes_neighbor_times_list[idx][-(max_input_sequence_length - 1):]
            if len(nodes_neighbor_ids_list[idx]) > max_seq_length:
                max_seq_length = len(nodes_neighbor_ids_list[idx])

        # include the target node itself
        max_seq_length += 1
        if max_seq_length % patch_size != 0:
            max_seq_length += (patch_size - max_seq_length % patch_size)
        assert max_seq_length % patch_size  == 0

        # pad the sequences
        # three ndarrays with shape (batch_size, max_seq_length)
        padded_nodes_neighbor_ids = np.zeros((len(node_ids), max_seq_length)).astype(np.long)
        padded_nodes_edge_ids = np.zeros((len(node_ids), max_seq_length)).astype(np.long)
        padded_nodes_neighbor_times = np.zeros((len(node_ids), max_seq_length)).astype(np.float32)

        for idx in range(len(node_ids)):
            padded_nodes_neighbor_ids[idx, 0] = node_ids[idx]
            padded_nodes_edge_ids[idx, 0] = 0
            padded_nodes_neighbor_times[idx, 0] = node_interact_times[idx]

            if len(nodes_neighbor_ids_list[idx]) > 0:
                padded_nodes_neighbor_ids[idx, 1: len(nodes_neighbor_ids_list[idx]) + 1] = nodes_neighbor_ids_list[idx]
                padded_nodes_edge_ids[idx, 1: len(nodes_edge_ids_list[idx]) + 1] = nodes_edge_ids_list[idx]
                padded_nodes_neighbor_times[idx, 1: len(nodes_neighbor_times_list[idx]) + 1] = nodes_neighbor_times_list[idx]

        # three ndarrays with shape (batch_size, max_seq_length)
        return padded_nodes_neighbor_ids, padded_nodes_edge_ids, padded_nodes_neighbor_times

    def get_features(self, node_interact_times: np.ndarray, padded_nodes_neighbor_ids: np.ndarray, padded_nodes_edge_ids: np.ndarray,
                     padded_nodes_neighbor_times: np.ndarray, time_encoder: TimeEncoder):
        """
        get node, edge and time features
        :param node_interact_times: ndarray, shape (batch_size, )
        :param padded_nodes_neighbor_ids: ndarray, shape (batch_size, max_seq_length)
        :param padded_nodes_edge_ids: ndarray, shape (batch_size, max_seq_length)
        :param padded_nodes_neighbor_times: ndarray, shape (batch_size, max_seq_length)
        :param time_encoder: TimeEncoder, time encoder
        :return:
        """
        # Tensor, shape (batch_size, max_seq_length, node_feat_dim)
        padded_nodes_neighbor_node_raw_features = self.node_raw_features[torch.from_numpy(padded_nodes_neighbor_ids)]
        # Tensor, shape (batch_size, max_seq_length, edge_feat_dim)
        padded_nodes_edge_raw_features = self.edge_raw_features[torch.from_numpy(padded_nodes_edge_ids)]
        # Tensor, shape (batch_size, max_seq_length, time_feat_dim)
        padded_nodes_neighbor_time_features = time_encoder(timestamps=torch.from_numpy(node_interact_times[:, np.newaxis] - padded_nodes_neighbor_times).float().to(self.device))

        # ndarray, set the time features to all zeros for the padded timestamp
        padded_nodes_neighbor_time_features[torch.from_numpy(padded_nodes_neighbor_ids == 0)] = 0.0

        return padded_nodes_neighbor_node_raw_features, padded_nodes_edge_raw_features, padded_nodes_neighbor_time_features

    def get_patches(self, padded_nodes_neighbor_node_raw_features: torch.Tensor, padded_nodes_edge_raw_features: torch.Tensor,
                    padded_nodes_neighbor_time_features: torch.Tensor, padded_nodes_neighbor_co_occurrence_features: torch.Tensor = None, patch_size: int = 1):
        """
        get the sequence of patches for nodes
        :param padded_nodes_neighbor_node_raw_features: Tensor, shape (batch_size, max_seq_length, node_feat_dim)
        :param padded_nodes_edge_raw_features: Tensor, shape (batch_size, max_seq_length, edge_feat_dim)
        :param padded_nodes_neighbor_time_features: Tensor, shape (batch_size, max_seq_length, time_feat_dim)
        :param padded_nodes_neighbor_co_occurrence_features: Tensor, shape (batch_size, max_seq_length, neighbor_co_occurrence_feat_dim)
        :param patch_size: int, patch size
        :return:
        """
        assert padded_nodes_neighbor_node_raw_features.shape[1] % patch_size == 0
        num_patches = padded_nodes_neighbor_node_raw_features.shape[1] // patch_size

        # list of Tensors with shape (num_patches, ), each Tensor with shape (batch_size, patch_size, node_feat_dim)
        patches_nodes_neighbor_node_raw_features, patches_nodes_edge_raw_features, \
        patches_nodes_neighbor_time_features, patches_nodes_neighbor_co_occurrence_features = [], [], [], []

        for patch_id in range(num_patches):
            start_idx = patch_id * patch_size
            end_idx = patch_id * patch_size + patch_size
            patches_nodes_neighbor_node_raw_features.append(padded_nodes_neighbor_node_raw_features[:, start_idx: end_idx, :])
            patches_nodes_edge_raw_features.append(padded_nodes_edge_raw_features[:, start_idx: end_idx, :])
            patches_nodes_neighbor_time_features.append(padded_nodes_neighbor_time_features[:, start_idx: end_idx, :])
            patches_nodes_neighbor_co_occurrence_features.append(padded_nodes_neighbor_co_occurrence_features[:, start_idx: end_idx, :])

        batch_size = len(padded_nodes_neighbor_node_raw_features)
        # Tensor, shape (batch_size, num_patches, patch_size * node_feat_dim)
        patches_nodes_neighbor_node_raw_features = torch.stack(patches_nodes_neighbor_node_raw_features, dim=1).reshape(batch_size, num_patches, patch_size * self.node_feat_dim)
        # Tensor, shape (batch_size, num_patches, patch_size * edge_feat_dim)
        patches_nodes_edge_raw_features = torch.stack(patches_nodes_edge_raw_features, dim=1).reshape(batch_size, num_patches, patch_size * self.edge_feat_dim)
        # Tensor, shape (batch_size, num_patches, patch_size * time_feat_dim)
        patches_nodes_neighbor_time_features = torch.stack(patches_nodes_neighbor_time_features, dim=1).reshape(batch_size, num_patches, patch_size * self.time_feat_dim)

        patches_nodes_neighbor_co_occurrence_features = torch.stack(patches_nodes_neighbor_co_occurrence_features, dim=1).reshape(batch_size, num_patches, patch_size * self.neighbor_co_occurrence_feat_dim)

        return patches_nodes_neighbor_node_raw_features, patches_nodes_edge_raw_features, patches_nodes_neighbor_time_features, patches_nodes_neighbor_co_occurrence_features

    def set_neighbor_sampler(self, neighbor_sampler: NeighborSampler):
        """
        set neighbor sampler to neighbor_sampler and reset the random state (for reproducing the results for uniform and time_interval_aware sampling)
        :param neighbor_sampler: NeighborSampler, neighbor sampler
        :return:
        """
        self.neighbor_sampler = neighbor_sampler
        if self.neighbor_sampler.sample_neighbor_strategy in ['uniform', 'time_interval_aware']:
            assert self.neighbor_sampler.seed is not None
            self.neighbor_sampler.reset_random_state()


class NeighborCooccurrenceEncoder(nn.Module):

    def __init__(self, neighbor_co_occurrence_feat_dim: int, device: str = 'cpu'):
        """
        Neighbor co-occurrence encoder.
        :param neighbor_co_occurrence_feat_dim: int, dimension of neighbor co-occurrence features (encodings)
        :param device: str, device
        """
        super(NeighborCooccurrenceEncoder, self).__init__()
        self.neighbor_co_occurrence_feat_dim = neighbor_co_occurrence_feat_dim
        self.device = device

        self.neighbor_co_occurrence_encode_layer = nn.Sequential(
            nn.Linear(in_features=1, out_features=self.neighbor_co_occurrence_feat_dim),
            nn.ReLU(),
            nn.Linear(in_features=self.neighbor_co_occurrence_feat_dim, out_features=self.neighbor_co_occurrence_feat_dim)).to(self.device)

    def count_nodes_appearances(self, src_padded_nodes_neighbor_ids: np.ndarray, dst_padded_nodes_neighbor_ids: np.ndarray):
        """
        count the appearances of nodes in the sequences of source and destination nodes
        :param src_padded_nodes_neighbor_ids: ndarray, shape (batch_size, src_max_seq_length)
        :param dst_padded_nodes_neighbor_ids:: ndarray, shape (batch_size, dst_max_seq_length)
        :return:
        """
        # two lists to store the appearances of source and destination nodes
        src_padded_nodes_appearances, dst_padded_nodes_appearances = [], []
        # src_padded_node_neighbor_ids, ndarray, shape (src_max_seq_length, )
        # dst_padded_node_neighbor_ids, ndarray, shape (dst_max_seq_length, )
        for src_padded_node_neighbor_ids, dst_padded_node_neighbor_ids in zip(src_padded_nodes_neighbor_ids, dst_padded_nodes_neighbor_ids):

            # src_unique_keys, ndarray, shape (num_src_unique_keys, )
            # src_inverse_indices, ndarray, shape (src_max_seq_length, )
            # src_counts, ndarray, shape (num_src_unique_keys, )
            # we can use src_unique_keys[src_inverse_indices] to reconstruct the original input, and use src_counts[src_inverse_indices] to get counts of the original input
            src_unique_keys, src_inverse_indices, src_counts = np.unique(src_padded_node_neighbor_ids, return_inverse=True, return_counts=True)
            # Tensor, shape (src_max_seq_length, )
            src_padded_node_neighbor_counts_in_src = torch.from_numpy(src_counts[src_inverse_indices]).float().to(self.device)
            # dictionary, store the mapping relation from unique neighbor id to its appearances for the source node
            src_mapping_dict = dict(zip(src_unique_keys, src_counts))

            # dst_unique_keys, ndarray, shape (num_dst_unique_keys, )
            # dst_inverse_indices, ndarray, shape (dst_max_seq_length, )
            # dst_counts, ndarray, shape (num_dst_unique_keys, )
            # we can use dst_unique_keys[dst_inverse_indices] to reconstruct the original input, and use dst_counts[dst_inverse_indices] to get counts of the original input
            dst_unique_keys, dst_inverse_indices, dst_counts = np.unique(dst_padded_node_neighbor_ids, return_inverse=True, return_counts=True)
            # Tensor, shape (dst_max_seq_length, )
            dst_padded_node_neighbor_counts_in_dst = torch.from_numpy(dst_counts[dst_inverse_indices]).float().to(self.device)
            # dictionary, store the mapping relation from unique neighbor id to its appearances for the destination node
            dst_mapping_dict = dict(zip(dst_unique_keys, dst_counts))

            # we need to use copy() to avoid the modification of src_padded_node_neighbor_ids
            # Tensor, shape (src_max_seq_length, )
            src_padded_node_neighbor_counts_in_dst = torch.from_numpy(src_padded_node_neighbor_ids.copy()).apply_(lambda neighbor_id: dst_mapping_dict.get(neighbor_id, 0.0)).float().to(self.device)
            # Tensor, shape (src_max_seq_length, 2)
            src_padded_nodes_appearances.append(torch.stack([src_padded_node_neighbor_counts_in_src, src_padded_node_neighbor_counts_in_dst], dim=1))

            # we need to use copy() to avoid the modification of dst_padded_node_neighbor_ids
            # Tensor, shape (dst_max_seq_length, )
            dst_padded_node_neighbor_counts_in_src = torch.from_numpy(dst_padded_node_neighbor_ids.copy()).apply_(lambda neighbor_id: src_mapping_dict.get(neighbor_id, 0.0)).float().to(self.device)
            # Tensor, shape (dst_max_seq_length, 2)
            dst_padded_nodes_appearances.append(torch.stack([dst_padded_node_neighbor_counts_in_src, dst_padded_node_neighbor_counts_in_dst], dim=1))

        # Tensor, shape (batch_size, src_max_seq_length, 2)
        src_padded_nodes_appearances = torch.stack(src_padded_nodes_appearances, dim=0)
        # Tensor, shape (batch_size, dst_max_seq_length, 2)
        dst_padded_nodes_appearances = torch.stack(dst_padded_nodes_appearances, dim=0)

        # set the appearances of the padded node (with zero index) to zeros
        # Tensor, shape (batch_size, src_max_seq_length, 2)
        src_padded_nodes_appearances[torch.from_numpy(src_padded_nodes_neighbor_ids == 0)] = 0.0
        # Tensor, shape (batch_size, dst_max_seq_length, 2)
        dst_padded_nodes_appearances[torch.from_numpy(dst_padded_nodes_neighbor_ids == 0)] = 0.0

        return src_padded_nodes_appearances, dst_padded_nodes_appearances

    def forward(self, src_padded_nodes_neighbor_ids: np.ndarray, dst_padded_nodes_neighbor_ids: np.ndarray):
        """
        compute the neighbor co-occurrence features of nodes in src_padded_nodes_neighbor_ids and dst_padded_nodes_neighbor_ids
        :param src_padded_nodes_neighbor_ids: ndarray, shape (batch_size, src_max_seq_length)
        :param dst_padded_nodes_neighbor_ids:: ndarray, shape (batch_size, dst_max_seq_length)
        :return:
        """
        # src_padded_nodes_appearances, Tensor, shape (batch_size, src_max_seq_length, 2)
        # dst_padded_nodes_appearances, Tensor, shape (batch_size, dst_max_seq_length, 2)
        src_padded_nodes_appearances, dst_padded_nodes_appearances = self.count_nodes_appearances(src_padded_nodes_neighbor_ids=src_padded_nodes_neighbor_ids,
                                                                                                  dst_padded_nodes_neighbor_ids=dst_padded_nodes_neighbor_ids)

        # sum the neighbor co-occurrence features in the sequence of source and destination nodes
        # Tensor, shape (batch_size, src_max_seq_length, neighbor_co_occurrence_feat_dim)
        src_padded_nodes_neighbor_co_occurrence_features = self.neighbor_co_occurrence_encode_layer(src_padded_nodes_appearances.unsqueeze(dim=-1)).sum(dim=2)
        # Tensor, shape (batch_size, dst_max_seq_length, neighbor_co_occurrence_feat_dim)
        dst_padded_nodes_neighbor_co_occurrence_features = self.neighbor_co_occurrence_encode_layer(dst_padded_nodes_appearances.unsqueeze(dim=-1)).sum(dim=2)

        # src_padded_nodes_neighbor_co_occurrence_features, Tensor, shape (batch_size, src_max_seq_length, neighbor_co_occurrence_feat_dim)
        # dst_padded_nodes_neighbor_co_occurrence_features, Tensor, shape (batch_size, dst_max_seq_length, neighbor_co_occurrence_feat_dim)
        return src_padded_nodes_neighbor_co_occurrence_features, dst_padded_nodes_neighbor_co_occurrence_features

This cell holds the definition of get_neighbor_sampler method in HOT encoder.\
The original method utilizes pandas DataFrame type arguments, but I modified it to be fed with numpy ndarray type ones.

In [ ]:
def get_neighbor_sampler(data, sample_neighbor_strategy: str, time_scaling_factor: float = 0.0, seed: int = None):
    """
    get neighbor sampler
    :param data: Data
    :param sample_neighbor_strategy: str, how to sample historical neighbors, 'uniform', 'recent', or 'time_interval_aware''
    :param time_scaling_factor: float, a hyper-parameter that controls the sampling preference with time interval,
    a large time_scaling_factor tends to sample more on recent links, this parameter works when sample_neighbor_strategy == 'time_interval_aware'
    :param seed: int, random seed
    :return:
    """
    max_node_id = max(data[:,0].max(), data[:,1].max())
    # the adjacency vector stores edges for each node (source or destination), undirected
    # adj_list, list of list, where each element is a list of triple tuple (node_id, edge_id, timestamp)
    # the list at the first position in adj_list is empty
    adj_list = [[] for _ in range(max_node_id + 1)]
    edge_ids=torch.as_tensor([0 for i in range(data.shape[0])])
    for src_node_id, dst_node_id, edge_id, node_interact_time in zip(data[:,0], data[:,1], edge_ids, data[2]):
        adj_list[src_node_id].append((dst_node_id, edge_id, node_interact_time))
        adj_list[dst_node_id].append((src_node_id, edge_id, node_interact_time))

    return NeighborSampler(adj_list=adj_list, sample_neighbor_strategy=sample_neighbor_strategy, time_scaling_factor=time_scaling_factor, seed=seed)


This cell holds my own code implementation of the Triadic Decoder, introduced in [Shi et al., 2020](https://arxiv.org/abs/1911.11322).

In [ ]:
class TriadicDecoder(nn.Module):
  def __init__(self, latentDim, numFilter, device):
    super(TriadicDecoder,self).__init__()
    self.latentDim=latentDim
    self.numFilter=numFilter
    self.device=device
    self.conv1=nn.Conv1d(in_channels=1, out_channels=self.numFilter, kernel_size=1).to(device)
    self.conv2=nn.Conv1d(in_channels=1, out_channels=self.numFilter, kernel_size=1).to(device)
    self.conv3=nn.Conv1d(in_channels=1, out_channels=self.numFilter, kernel_size=1).to(device)
    self.relu=nn.ReLU().to(device)
    self.ss=nn.Softsign().to(device)
    self.BN=nn.BatchNorm1d(num_features=self.latentDim).to(device)
    self.LN=nn.LayerNorm(normalized_shape=self.latentDim).to(device)
    self.dropout=nn.Dropout(p=0.1).to(device)
    # FC layers - how they transformed (1, 4, 116) z_triplet into (1, 1, 3) is not specified
    # Therefore I applied 2 FC layers without placing nonlinearities in between
    self.fc1=nn.Linear(in_features=self.latentDim, out_features=3).to(device)
    self.convFinal=nn.Conv1d(in_channels=self.numFilter, out_channels=1, kernel_size=1).to(device)


  def forward(self, embed1, embed2, embed3):
    IP=torch.zeros((3,)).to(self.device)
    embed1=torch.squeeze(embed1, dim=0)
    embed2=torch.squeeze(embed2, dim=0)
    embed3=torch.squeeze(embed3, dim=0)
    norm1=torch.sqrt(torch.sum(embed1*embed1)+1e-10)
    norm2=torch.sqrt(torch.sum(embed2*embed2)+1e-10)
    norm3=torch.sqrt(torch.sum(embed3*embed3)+1e-10)
    embed1=embed1/norm1
    embed2=embed2/norm2
    embed3=embed3/norm3

    IP[0]=torch.dot(embed1, embed2)
    IP[1]=torch.dot(embed1, embed3)
    IP[2]=torch.dot(embed2, embed3)
    v1=self.conv1(embed1.unsqueeze(0))
    v2=self.conv2(embed2.unsqueeze(0))
    v3=self.conv3(embed3.unsqueeze(0))
    zTriplet=self.relu(v1+v2+v3) # (4, 116)
    w=self.fc1(zTriplet) # (4, 3)
    x=self.convFinal(w) # (1, 3)
    y=self.relu(x)
    z=torch.sigmoid(y+IP).squeeze(0)
    return z

'\nclass EmbeddingGenerator(nn.Module):\n  def __init__(self, inputFeat, hiddenFeat, embedFeat, device):\n    super(EmbeddingGenerator, self).__init__()\n    self.device=device\n    self.inputFeat=inputFeat\n    self.embedFeat=embedFeat\n    self.hiddenFeat=hiddenFeat\n    self.layer1=nn.Linear(in_features=self.inputFeat, out_features=self.hiddenFeat).to(device)\n    self.layer2=nn.Linear(in_features=self.hiddenFeat, out_features=self.embedFeat).to(device)\n    self.relu=nn.ReLU()\n    self.ss=nn.Softsign()\n    self.elu=nn.ELU()\n  def forward(self, embed):\n    return self.layer2(self.ss(self.layer1(embed)))\n'

# Training Helper functions

This cell contains the definitions of trianing helper functions.

In [ ]:
def sampleFrom(from1, from2, p, device):
  sampleFrom1=torch.bernoulli(torch.tensor([p])).to(device)
  if sampleFrom1:
    idx = torch.randint(len(from1), (1,)).item()
    return from1[idx]
  else:
    idx = torch.randint(len(from2), (1,)).item()
    return from2[idx]

# Parameter B is equivalent to the number of triads being sampled
def triadSample(triplets, B, p, device):
  triadList=[]
  i=0
  nodes=torch.Tensor([i for i in range(116)]).to(device)
  while i<B:
    node1=torch.randperm(116)[0].to(device)
    idx1=((triplets[:,0]==node1).nonzero())[:,0].to(device)
    neighbors1=triplets[idx1,1]
    mask1 = torch.isin(nodes, neighbors1, invert=True).to(device)
    faraways = nodes[mask1].to(device)
    # Have to remove node1 from faraways1
    temp=(faraways==node1).nonzero()
    if len(temp)>0:
      continue
    node2=sampleFrom(neighbors1, faraways, p, device).to(device)

    idx2=((triplets[:,0]==node2).nonzero())[:,0].to(device)
    neighbors2=triplets[idx2,1]
    mask2=torch.isin(nodes, neighbors2, invert=True).to(device)
    faraways = nodes[mask2].to(device)
    # Have to remove node1 from faraways1
    temp=(faraways==node2).nonzero()
    if len(temp)>0:
      continue
    node3=sampleFrom(neighbors2, faraways, p, device).to(device)
    triadList.append(torch.Tensor([node1, node2, node3]).to(device))
    i+=1


  triads=torch.stack(triadList, dim=0).int().to(device)
  assert triads.shape==(B, 3)
  return triads

def calculateM(triads, pair):
  pair=pair.squeeze(0)
  assert pair.shape==(2,)
  assert triads.shape[1]==3
  occurrence=0
  if pair[0] in triads[:,0]:
    idx1=((triads[:,0]==pair[0]).nonzero())[:,0]
    if pair[1] in triads[idx1,1]:
      occurrence+=((pair[1]==triads[idx1,1]).nonzero())[:,0].shape[0]
    if pair[1] in triads[idx1,2]:
      occurrence+=((pair[1]==triads[idx1,2]).nonzero())[:,0].shape[0]
  if pair[0] in triads[:,1]:
    idx2=((triads[:,1]==pair[0]).nonzero())[:,0]
    if pair[1] in triads[idx2,2]:
      occurrence+=((pair[1]==triads[idx2,2]).nonzero())[:,0].shape[0]
  return occurrence

def check_submodules_have_attributes(model):
  dic={}
  for name, submodule in model.named_modules():
      # Skip the top-level model itself
      if name == "":
          continue

      has_parameters_attr = hasattr(submodule, 'parameters')
      has_weight_attr = hasattr(submodule, 'weight')
      if has_parameters_attr:
        has_gradient_attr = hasattr(submodule.parameters(), 'requires_grad')
      #print(f"Submodule '{name}' has 'parameters' attribute: {has_parameters_attr}")
      #print(f"Submodule '{name}' has 'weight' attribute: {has_weight_attr}")
      dic[name]=(has_parameters_attr, has_weight_attr, has_gradient_attr)
  return dic

def init_kaiming_normal(model):
  check=check_submodules_have_attributes(model)
  for name, submodule in model.named_modules():
    if name == "":
      continue
    if check[name]==(True, True, True):
      torch.nn.init.kaiming_normal_(submodule.weight)
      torch.nn.init.zeros_(submodule.bias)


def pairIn(pair, duets):
  if pair[0] in duets[:,0]:
    idx1=((duets[:,0]==pair[0]).nonzero())[:,0]
    if pair[1] in duets[idx1,1]:
      return True

# HOT Encoder + Triadic Decoder - Experiment



The cell below is the training and validation loop for 40th Quantile Dataset experiment. \
The model will be saved in "checkpoint8.pth" file.

In [ ]:
# 40th Quantile Dataset Experiment

torch.autograd.set_detect_anomaly(False)
if os.path.isfile('checkpoint8.pth'):
        print('resuming checkpoint experiment')
        checkpoint = torch.load('checkpoint8.pth', map_location='cuda')
else:
  checkpoint = {
    'epoch': 0,
    'subject': 0,
    'timeIdx':0,
    'encoder': None,
    'decoder': None,
    'optimizerE': None,
    'lossSubject': None,
    'prevLoss' : np.Inf,
    'patience': 0,
    'lossSubjectV': np.Inf,
    'lr': 1e-4
    }

# Argv setting
epc=10
sampleStrategy='uniform'
negSampleStrategy='inductive'
N=116 # Number of ROIs
numEdges=math.ceil(116*116*0.4)
nodeDim=N # One-Hot encoding as a node embedding
edgeDim=4 # Arbitrarily set
timeDim=100 # Half of the value mentioned in the HOT paper
channelDim=50 # The value mentioned in the HOT paper
latentDim=N # Positive node embedding and negative node embedding concatenated across dim=1
numFilter=4 # Following Triadic Decoder paper
B=10000
prob=0.7398
nodeFeat=torch.cat((torch.eye(N),torch.zeros((1,N))),dim=0).numpy()
edgeFeat=torch.zeros((numEdges+1,edgeDim)).numpy()
learning_rateE=1e-4
pt=2
patience=0
prevLoss=np.inf
lossSubjectV=np.inf


encoder=HOT(nodeFeat, edgeFeat, None, timeDim, channelDim, patch_size=8, num_state_vectors=32, num_layers=2, num2hop=0, dropout=0.1, max_input_sequence_length=4096, device='cuda')
encoder.apply(init_kaiming_normal)
decoder=TriadicDecoder(latentDim, numFilter, device='cuda')
decoder.apply(init_kaiming_normal)
model=nn.Sequential(encoder, decoder).to('cuda')
lossFunc=nn.BCELoss().to('cuda')
optimizerE=torch.optim.Adam(model.parameters(), lr=checkpoint['lr'])


for epoch in range(checkpoint['epoch'], epc):
  print("Epoch : "+str(epoch))
  if os.path.isfile('checkpoint8.pth'):
    checkpoint=torch.load('checkpoint8.pth', map_location='cuda')
  for i in range(checkpoint['subject'], 16):
    if os.path.isfile('checkpoint8.pth'):
      checkpoint=torch.load('checkpoint8.pth', map_location='cuda')
    model.train()
    if checkpoint['lr']!=0: learning_rateE=checkpoint['lr']
    if checkpoint['optimizerE'] is not None: optimizerE.load_state_dict(checkpoint['optimizerE'])
    if checkpoint['prevLoss'] is not None: prevLoss=checkpoint['prevLoss']
    if checkpoint['patience'] != 0: patience=checkpoint['patience']
    if checkpoint['lossSubjectV'] is not None: lossSubjectV=checkpoint['lossSubjectV']
    if checkpoint['encoder'] is not None: model[0].load_state_dict(checkpoint['encoder'])
    if checkpoint['decoder'] is not None: model[1].load_state_dict(checkpoint['decoder'])
    print("\nSubject index : "+str(i)+" out of 16")

    train3D=torch.load(os.path.join('DatasetSplit','train'+str(i)+'.pth'))
    train3D=train3D.reshape(-1,3).detach().to('cpu').numpy()
    train3D=train3D.reshape(15, -1, 3)

    print("Sampling probability : "+str(prob))

    lossSubject=torch.zeros(1).to('cuda')
    lossSubject.requires_grad_()

    for t in range(checkpoint['timeIdx'], 15):
      print("Time index : "+str(t)+" out of 15")

      timeBatch=train3D[t]
      srcNodes=timeBatch[:,0]
      dstNodes=timeBatch[:,1]
      timepoints=timeBatch[:,2]

      adjMat=torch.zeros((N,N)).to('cuda')
      for row in timeBatch:
        adjMat[int(row[0]), int(row[1])]=1

      nodePosEmbedding=torch.zeros((N,N)).to('cuda')
      nodeNegEmbedding=torch.zeros((N,N)).to('cuda')
      nodePosTimes=torch.zeros(N).to('cuda')
      nodeNegTimes=torch.zeros(N).to('cuda')
      nodeEmbedding=torch.zeros((N,N)).to('cuda')

      # Sample neighbor edges among the dynamic graphs of a single subject
      train_neighbor_sampler = get_neighbor_sampler(data=timeBatch, sample_neighbor_strategy=sampleStrategy, seed=0)
      # Sample negative edges among the dynamic graphs of a single subject
      train_neg_edge_sampler = NegativeEdgeSampler(src_node_ids=srcNodes, dst_node_ids=dstNodes, interact_times=timepoints, last_observed_time=np.min(timepoints) , negative_sample_strategy=negSampleStrategy, seed=1)
      model[0].set_neighbor_sampler(train_neighbor_sampler)

      subject_src_node_embeddings, subject_dst_node_embeddings = \
        torch.utils.checkpoint.checkpoint(model[0].compute_src_dst_node_temporal_embeddings, srcNodes, dstNodes, timepoints, use_reentrant=False)
      negSrcNodes, negDstNodes = train_neg_edge_sampler.sample(len(srcNodes), srcNodes, dstNodes, np.min(timepoints), np.max(timepoints))
      subject_neg_src_node_embeddings, subject_neg_dst_node_embeddings = \
        torch.utils.checkpoint.checkpoint(model[0].compute_src_dst_node_temporal_embeddings, negSrcNodes, negDstNodes, timepoints, use_reentrant=False)

      for tb, row in enumerate(zip(srcNodes, dstNodes)):
        nodePosEmbedding[row[0]]=nodePosEmbedding[row[0]]+subject_src_node_embeddings[tb]
        nodePosTimes[row[0]]+=1
        nodePosEmbedding[row[1]]=nodePosEmbedding[row[1]]+subject_dst_node_embeddings[tb]
        nodePosTimes[row[1]]+=1
      for tb, row in enumerate(zip(negSrcNodes, negDstNodes)):
        nodeNegEmbedding[row[0]]=nodeNegEmbedding[row[0]]+subject_neg_src_node_embeddings[tb]
        nodeNegTimes[row[0]]+=1
        nodeNegEmbedding[row[1]]=nodeNegEmbedding[row[1]]+subject_neg_dst_node_embeddings[tb]
        nodeNegTimes[row[1]]+=1

      for node in range(N):
        if nodePosTimes[node]>0:
          nodePosEmbedding[node]=nodePosEmbedding[node]/nodePosTimes[node]
        if nodeNegTimes[node]>0:
          nodeNegEmbedding[node]=nodeNegEmbedding[node]/nodeNegTimes[node]


      nodePosEmbedding=(nodePosEmbedding-torch.mean(nodePosEmbedding, dim=0, keepdims=True))/(torch.std(nodePosEmbedding, dim=0, keepdims=True)+1e-10)
      nodeNegEmbedding=(nodeNegEmbedding-torch.mean(nodeNegEmbedding, dim=0, keepdims=True))/(torch.std(nodeNegEmbedding, dim=0, keepdims=True)+1e-10)

      graph2D=np.concatenate((np.expand_dims(srcNodes, axis=1), np.expand_dims(dstNodes, axis=1)), axis=1)
      graph2D=torch.tensor(graph2D).to('cuda')
      graph2DN=np.concatenate((np.expand_dims(negSrcNodes, axis=1), np.expand_dims(negDstNodes, axis=1)), axis=1)
      graph2DN=torch.tensor(graph2DN).to('cuda')
      triads=triadSample(graph2D, B, prob, device='cuda')

      mSquare=torch.zeros((N,N)).to('cuda')
      eSquare=torch.zeros((N,N)).to('cuda')

      for j, triad in enumerate(triads):
        triad0=triad[0].unsqueeze(0).unsqueeze(0)
        triad1=triad[1].unsqueeze(0).unsqueeze(0)
        triad2=triad[2].unsqueeze(0).unsqueeze(0)

        if mSquare[triad[0], triad[1]]==0:
          pair1=torch.cat((triad0, triad1), dim=1).to('cuda')
          M1=calculateM(triads, pair1)
          mSquare[triad[0], triad[1]]=M1
        if mSquare[triad[0], triad[2]]==0:
          pair2=torch.cat((triad0, triad2), dim=1).to('cuda')
          M2=calculateM(triads, pair2)
          mSquare[triad[0], triad[2]]=M2
        if mSquare[triad[1], triad[2]]==0:
          pair3=torch.cat((triad1, triad2), dim=1).to('cuda')
          M3=calculateM(triads, pair3)
          mSquare[triad[1], triad[2]]=M3

      for j, triad in enumerate(triads):
        triad0=triad[0].unsqueeze(0).unsqueeze(0)
        triad1=triad[1].unsqueeze(0).unsqueeze(0)
        triad2=triad[2].unsqueeze(0).unsqueeze(0)
        pair1=torch.cat((triad0, triad1), dim=1).squeeze(0).to('cuda')
        pair2=torch.cat((triad0, triad2), dim=1).squeeze(0).to('cuda')
        pair3=torch.cat((triad1, triad2), dim=1).squeeze(0).to('cuda')

        emb1=torch.zeros((1,N)).to('cuda')
        emb2=torch.zeros((1,N)).to('cuda')
        emb3=torch.zeros((1,N)).to('cuda')

        if pairIn(pair1,graph2D[:,0:2]):
          emb1=nodePosEmbedding[triad[0]]
        if pairIn(pair1,graph2DN):
          emb1=nodeNegEmbedding[triad[0]]
        if pairIn(pair2,graph2D[:,0:2]):
          emb2=nodePosEmbedding[triad[1]]
        if pairIn(pair2,graph2DN):
          emb2=nodeNegEmbedding[triad[1]]
        if pairIn(pair3,graph2D[:,0:2]):
          emb3=nodePosEmbedding[triad[2]]
        if pairIn(pair3,graph2DN):
          emb3=nodeNegEmbedding[triad[2]]

        eTriplet=model[1].forward(emb1, emb2, emb3)
        if pairIn(pair1,graph2D[:,0:2]):
          eSquare[triad[0], triad[1]]+=eTriplet[0]
        if pairIn(pair1,graph2DN):
          eSquare[triad[0], triad[1]]-=eTriplet[0]
        if pairIn(pair2,graph2D[:,0:2]):
          eSquare[triad[0], triad[2]]+=eTriplet[1]
        if pairIn(pair2,graph2DN):
          eSquare[triad[0], triad[2]]-=eTriplet[1]
        if pairIn(pair3,graph2D[:,0:2]):
          eSquare[triad[1], triad[2]]+=eTriplet[2]
        if pairIn(pair3,graph2DN):
          eSquare[triad[1], triad[2]]-=eTriplet[2]
      eSquare.clamp_(min=0.0)

      nonzeros=((mSquare>0).nonzero())
      adjRecon=[]
      adjPart=[]
      adjFull=torch.zeros((N,N)).to('cuda')
      for nz in nonzeros:
        nzPair=torch.tensor([int(nz[0]), int(nz[1])]).to('cuda')
        em1=eSquare[int(nz[0]), int(nz[1])]/mSquare[int(nz[0]), int(nz[1])]
        if pairIn(nzPair,graph2D[:,0:2]):
          adjRecon.append(em1)
          adjPart.append(torch.tensor([1.0]))
        if pairIn(nzPair, graph2DN):
          adjRecon.append(em1)
          adjPart.append(torch.tensor([0.0]))
        if em1>0.5:
          adjFull[int(nz[0]), int(nz[1])]=1
        else:
          adjFull[int(nz[0]), int(nz[1])]=0
      adjRecon=torch.stack(adjRecon).to('cuda')

      adjPart=torch.stack(adjPart).squeeze(1).to('cuda')


      loss=lossFunc(input=adjRecon, target=adjPart)
      lossSubject=lossSubject+loss

      adjReconC=adjRecon.clone().detach().cpu().numpy()
      adjPartC=adjPart.clone().detach().cpu().numpy()

      adjReconC[adjReconC>0.5]=1
      adjReconC[adjReconC<=0.5]=0
      F1=f1_score(adjPartC, adjReconC)
      AUROC=roc_auc_score(adjPartC, adjReconC)
      AP=average_precision_score(adjPartC, adjReconC)


      print("Epoch : "+str(epoch)+" / Subject : "+str(i)+" / Timepoint : "+str(t) +" / Loss : "+str(loss.item())+" / AUROC : "+str(AUROC)+" / F1 : "+str(F1)+" / AP : "+str(AP)+"\n")

      torch.cuda.empty_cache()

    optimizerE.zero_grad()
    lossSubject.backward()
    optimizerE.step()




    # Validation Step
    if (i+1)%2==0:
      model.eval()
      with torch.no_grad():
        patience=checkpoint['patience']
        prevLoss=checkpoint['prevLoss']
        valid3D=torch.load(os.path.join('DatasetSplit','valid'+str((i+1)//2-1)+'.pth'))
        valid3D=valid3D.reshape(-1, 3).detach().to('cpu').numpy()
        valid3D=valid3D.reshape(15, -1, 3)
        lossSubjectV=0

        print("Validation Subject index : "+str((i+1)//2-1)+" out of 8")
        for t in range(15):
          adjRecon=torch.zeros((N,N)).to('cuda')

          timeBatch=valid3D[t]
          timeBatch[:,2]=timeBatch[:,2]/3-1
          srcNodes=timeBatch[:,0]
          dstNodes=timeBatch[:,1]
          timepoints=timeBatch[:,2]

          adjMat=torch.zeros((N,N)).to('cuda')
          for row in timeBatch:
            adjMat[int(row[0]), int(row[1])]=1

          nodePosEmbedding=torch.zeros((N,N)).to('cuda')
          nodeNegEmbedding=torch.zeros((N,N)).to('cuda')
          nodePosTimes=torch.zeros(N).to('cuda')
          nodeNegTimes=torch.zeros(N).to('cuda')
          nodeEmbedding=torch.zeros((N,N)).to('cuda')

          # Sample neighbor edges among the dynamic graphs of a single subject
          valid_neighbor_sampler = get_neighbor_sampler(data=timeBatch, sample_neighbor_strategy=sampleStrategy, seed=2)
          # Sample negative edges among the dynamic graphs of a single subject
          valid_neg_edge_sampler = NegativeEdgeSampler(src_node_ids=srcNodes, dst_node_ids=dstNodes, interact_times=timepoints, last_observed_time=np.min(timepoints), negative_sample_strategy=negSampleStrategy, seed=3)
          model[0].set_neighbor_sampler(valid_neighbor_sampler)

          subject_src_node_embeddings, subject_dst_node_embeddings = \
            model[0].compute_src_dst_node_temporal_embeddings(src_node_ids=srcNodes, dst_node_ids=dstNodes, node_interact_times=timepoints)
          negSrcNodes, negDstNodes = valid_neg_edge_sampler.sample(len(srcNodes), srcNodes, dstNodes, np.min(timepoints), np.max(timepoints))
          subject_neg_src_node_embeddings, subject_neg_dst_node_embeddings = \
            model[0].compute_src_dst_node_temporal_embeddings(src_node_ids=negSrcNodes,
                                                                          dst_node_ids=negDstNodes,
                                                                          node_interact_times=timepoints)

          for tb, row in enumerate(zip(srcNodes, dstNodes)):
            nodePosEmbedding[row[0]]=nodePosEmbedding[row[0]]+subject_src_node_embeddings[tb]
            nodePosTimes[row[0]]+=1
            nodePosEmbedding[row[1]]=nodePosEmbedding[row[1]]+subject_dst_node_embeddings[tb]
            nodePosTimes[row[1]]+=1
          for tb, row in enumerate(zip(negSrcNodes, negDstNodes)):
            nodeNegEmbedding[row[0]]=nodeNegEmbedding[row[0]]+subject_neg_src_node_embeddings[tb]
            nodeNegTimes[row[0]]+=1
            nodeNegEmbedding[row[1]]=nodeNegEmbedding[row[1]]+subject_neg_dst_node_embeddings[tb]
            nodeNegTimes[row[1]]+=1

          for node in range(N):
            if nodePosTimes[node]>0:
              nodePosEmbedding[node]=nodePosEmbedding[node]/nodePosTimes[node]
            if nodeNegTimes[node]>0:
              nodeNegEmbedding[node]=nodeNegEmbedding[node]/nodeNegTimes[node]

          nodePosEmbedding=(nodePosEmbedding-torch.mean(nodePosEmbedding, dim=0, keepdims=True))/(torch.std(nodePosEmbedding, dim=0, keepdims=True)+1e-10)
          nodeNegEmbedding=(nodeNegEmbedding-torch.mean(nodeNegEmbedding, dim=0, keepdims=True))/(torch.std(nodeNegEmbedding, dim=0, keepdims=True)+1e-10)

          graph2D=np.concatenate((np.expand_dims(srcNodes, axis=1), np.expand_dims(dstNodes, axis=1)), axis=1)
          graph2D=torch.tensor(graph2D).to('cuda')
          graph2DN=np.concatenate((np.expand_dims(negSrcNodes, axis=1), np.expand_dims(negDstNodes, axis=1)), axis=1)
          graph2DN=torch.tensor(graph2DN).to('cuda')

          triads=triadSample(graph2D, B, prob, device='cuda')
          mSquare=torch.zeros((N,N)).to('cuda')
          eSquare=torch.zeros((N,N)).to('cuda')

          for j, triad in enumerate(triads):
            triad0=triad[0].unsqueeze(0).unsqueeze(0)
            triad1=triad[1].unsqueeze(0).unsqueeze(0)
            triad2=triad[2].unsqueeze(0).unsqueeze(0)

            if mSquare[triad[0], triad[1]]==0:
              pair1=torch.cat((triad0, triad1), dim=1).to('cuda')
              M1=calculateM(triads, pair1)
              mSquare[triad[0], triad[1]]=M1

            if mSquare[triad[0], triad[2]]==0:
              pair1=torch.cat((triad0, triad2), dim=1).to('cuda')
              M2=calculateM(triads, pair1)
              mSquare[triad[0], triad[2]]=M2

            if mSquare[triad[1], triad[2]]==0:
              pair1=torch.cat((triad1, triad2), dim=1).to('cuda')
              M3=calculateM(triads, pair1)
              mSquare[triad[1], triad[2]]=M3

          for j, triad in enumerate(triads):
            triad0=triad[0].unsqueeze(0).unsqueeze(0)
            triad1=triad[1].unsqueeze(0).unsqueeze(0)
            triad2=triad[2].unsqueeze(0).unsqueeze(0)
            pair1=torch.cat((triad0, triad1), dim=1).squeeze(0).to('cuda')
            pair2=torch.cat((triad0, triad2), dim=1).squeeze(0).to('cuda')
            pair3=torch.cat((triad1, triad2), dim=1).squeeze(0).to('cuda')

            if pair1 in graph2D[:,0:2]:
              emb1=nodePosEmbedding[triad[0]]
            if pair1 in graph2DN:
              emb1=nodeNegEmbedding[triad[0]]
            if pair2 in graph2D[:,0:2]:
              emb2=nodePosEmbedding[triad[1]]
            if pair2 in graph2DN:
              emb2=nodeNegEmbedding[triad[1]]
            if pair3 in graph2D[:,0:2]:
              emb3=nodePosEmbedding[triad[2]]
            if pair3 in graph2DN:
              emb3=nodeNegEmbedding[triad[2]]

            eTriplet=model[1].forward(emb1, emb2, emb3)
            if pairIn(pair1,graph2D[:,0:2]):
              eSquare[triad[0], triad[1]]+=eTriplet[0]
            if pairIn(pair1,graph2DN):
              eSquare[triad[0], triad[1]]-=eTriplet[0]
            if pairIn(pair2,graph2D[:,0:2]):
              eSquare[triad[0], triad[2]]+=eTriplet[1]
            if pairIn(pair2,graph2DN):
              eSquare[triad[0], triad[2]]-=eTriplet[1]
            if pairIn(pair3,graph2D[:,0:2]):
              eSquare[triad[1], triad[2]]+=eTriplet[2]
            if pairIn(pair3,graph2DN):
              eSquare[triad[1], triad[2]]-=eTriplet[2]

          eSquare.clamp_(min=0.0)
          nonzeros=((mSquare>0).nonzero())
          adjRecon=[]
          adjPart=[]
          adjFull=torch.zeros((N,N)).to('cuda')
          for nz in nonzeros:
            nzPair=torch.tensor([int(nz[0]), int(nz[1])]).to('cuda')
            em1=eSquare[int(nz[0]), int(nz[1])]/mSquare[int(nz[0]), int(nz[1])]
            if pairIn(nzPair, graph2D[:,0:2]):
              adjRecon.append(em1)
              adjPart.append(torch.tensor([1.0]))
            if pairIn(nzPair,graph2DN):
              adjRecon.append(em1)
              adjPart.append(torch.tensor([0.0]))
            if em1>0.5:
              adjFull[int(nz[0]), int(nz[1])]=1
            else:
              adjFull[int(nz[0]), int(nz[1])]=0
          adjRecon=torch.stack(adjRecon).to('cuda')
          adjPart=torch.stack(adjPart).squeeze(1).to('cuda')

          loss=lossFunc(input=adjRecon, target=adjPart)
          lossSubjectV=lossSubjectV+loss

          adjReconC=adjRecon.clone().detach().cpu().numpy()
          adjPartC=adjPart.clone().detach().cpu().numpy()

          adjReconC[adjReconC>0.5]=1
          adjReconC[adjReconC<=0.5]=0
          F1=f1_score(adjPartC, adjReconC)
          AUROC=roc_auc_score(adjPartC, adjReconC)
          AP=average_precision_score(adjPartC, adjReconC)

          print("Validation / Subject : "+str((i+1)//2-1)+" / Timepoint : "+str(t) +" / Loss : "+str(loss.item())+" / AP : "+str(AP)+" / F1 : "+str(F1)+" / AUROC : "+str(AUROC)+'\n')


        if lossSubjectV>prevLoss:
          patience+=1
        else:
          patience=0

        if patience==1 and learning_rateE<=2.5e-5:
          torch.save({
          'epoch': epoch,
          'encoder': model[0].state_dict(),
          'decoder': model[1].state_dict(),
          'subject': i,
          'timeIdx': t,
          'optimizerE': optimizerE.state_dict(),
          'lossSubject': lossSubject,
          'patience': patience,
          'prevLoss': prevLoss,
          'lossSubjectV': lossSubjectV,
          'lr': learning_rateE
          },
          'checkpoint8.pth')
          raise Exception('Early Stopping')
        if patience==pt and learning_rateE>2.5e-5:
          learning_rateE=learning_rateE/2
          patience=0

    if lossSubjectV is not None:
      prevLoss=lossSubjectV
    torch.save({
        'epoch': epoch,
        'encoder': model[0].state_dict(),
        'decoder': model[1].state_dict(),
        'subject': i+1,
        'timeIdx': 0,
        'optimizerE': optimizerE.state_dict(),
        'lossSubject': None,
        'patience': patience,
        'prevLoss': prevLoss,
        'lossSubjectV': lossSubjectV,
        'lr': learning_rateE
        },
        'checkpoint8.pth')

    torch.cuda.empty_cache()

  torch.save({
              'epoch': epoch+1,
              'encoder': model[0].state_dict(),
              'decoder': model[1].state_dict(),
              'subject': 0,
              'timeIdx': 0,
              'optimizerE': optimizerE.state_dict(),
              'lossSubject': None,
              'patience': patience,
              'prevLoss': prevLoss,
              'lossSubjectV': lossSubjectV,
              'lr': learning_rateE
              },
              'checkpoint8.pth')


resuming checkpoint experiment
Epoch : 1


<ipython-input-10-55ef187a96a8>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint8.pth', map_location='cuda')
<ipython-input-10-55ef187a96


Subject index : 3 out of 16
Sampling probability : 0.7398
Time index : 0 out of 15
tensor(0.3844, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 5, 37, 40, 41, 87, 88], device='cuda:0')
Epoch : 1 / Subject : 3 / Timepoint : 0 / Loss : 0.22708791494369507 / AUROC : 0.9818903738136742 / F1 : 0.9815563665055725 / AP : 0.9844232026444585

Time index : 1 out of 15
tensor(0.3866, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 35,  37,  40,  41,  52,  87, 106], device='cuda:0')
Epoch : 1 / Subject : 3 / Timepoint : 1 / Loss : 0.22421588003635406 / AUROC : 0.9831252438548577 / F1 : 0.9828355987697193 / AP : 0.9854983737800311

Time index : 2 out of 15
tensor(0.3892, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([  5,  21,  37,  38,  40,  52, 106], device='cuda:0')
Epoch : 1 / Subject : 3 / Timepoint : 2 / Loss : 0.21966208517551422 / AUROC : 0.9799567695028493 / F1 : 0.979546821736515 / AP : 0.9827476558626496

Time index : 3 out of 15
tensor(0.3877, device='cuda:0', grad_fn=<MeanB

<ipython-input-10-55ef187a96a8>:258: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  valid3D=torch.load(os.path.join('DatasetSplit','valid'+str((i+1)//2-1)+'.pth'))


Validation Subject index : 1 out of 8
Validation / Subject : 1 / Timepoint : 0 / Loss : 0.21057777106761932 / AP : 0.9906430730117177 / F1 : 0.9888866838658464 / AUROC : 0.9890088321884201

Validation / Subject : 1 / Timepoint : 1 / Loss : 0.21078670024871826 / AP : 0.9912093862304165 / F1 : 0.989517819706499 / AUROC : 0.9896265560165975

Validation / Subject : 1 / Timepoint : 2 / Loss : 0.20829203724861145 / AP : 0.9902741060565458 / F1 : 0.9884965489646894 / AUROC : 0.9886273734177216

Validation / Subject : 1 / Timepoint : 3 / Loss : 0.21202948689460754 / AP : 0.9888648981999949 / F1 : 0.9866640127388535 / AUROC : 0.9868395207228442

Validation / Subject : 1 / Timepoint : 4 / Loss : 0.2074330598115921 / AP : 0.9887732489858675 / F1 : 0.9867463876432486 / AUROC : 0.9869197482297404

Validation / Subject : 1 / Timepoint : 5 / Loss : 0.21078164875507355 / AP : 0.9880540754843059 / F1 : 0.9856411286273722 / AUROC : 0.9858443872500495

Validation / Subject : 1 / Timepoint : 6 / Loss : 0.

...

<ipython-input-10-55ef187a96a8>:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train3D=torch.load(os.path.join('DatasetSplit','train'+str(i)+'.pth'))


tensor(0.4114, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([36, 40, 86], device='cuda:0')
Epoch : 2 / Subject : 15 / Timepoint : 0 / Loss : 0.21998989582061768 / AUROC : 0.9799404170804369 / F1 : 0.9795297932711796 / AP : 0.9835981055005475

Time index : 1 out of 15
tensor(0.4106, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([40], device='cuda:0')
Epoch : 2 / Subject : 15 / Timepoint : 1 / Loss : 0.22080931067466736 / AUROC : 0.9758497316636852 / F1 : 0.9752520623281393 / AP : 0.9802306362755915

Time index : 2 out of 15
tensor(0.4094, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([40], device='cuda:0')
Epoch : 2 / Subject : 15 / Timepoint : 2 / Loss : 0.22485992312431335 / AUROC : 0.9735486427580742 / F1 : 0.9728299582782131 / AP : 0.9784316359445017

Time index : 3 out of 15
tensor(0.4031, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([40], device='cuda:0')
Epoch : 2 / Subject : 15 / Timepoint : 3 / Loss : 0.22104163467884064 / AUROC : 0.9747364233141038 / F1 : 0.97408

<ipython-input-10-55ef187a96a8>:258: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  valid3D=torch.load(os.path.join('DatasetSplit','valid'+str((i+1)//2-1)+'.pth'))


Validation Subject index : 7 out of 8
Validation / Subject : 7 / Timepoint : 0 / Loss : 0.196870818734169 / AP : 0.9920491237060474 / F1 : 0.9902719872940242 / AUROC : 0.9903657097915848

Validation / Subject : 7 / Timepoint : 1 / Loss : 0.19023564457893372 / AP : 0.9953721317271603 / F1 : 0.9944510503369005 / AUROC : 0.994481671265274

Validation / Subject : 7 / Timepoint : 2 / Loss : 0.18991710245609283 / AP : 0.9963322371343107 / F1 : 0.995583472372166 / AUROC : 0.9956028923197187

Validation / Subject : 7 / Timepoint : 3 / Loss : 0.19373288750648499 / AP : 0.9925073595979679 / F1 : 0.9909892068521636 / AUROC : 0.9910696761530913

Validation / Subject : 7 / Timepoint : 4 / Loss : 0.19824828207492828 / AP : 0.9900153042974286 / F1 : 0.9878461844989042 / AUROC : 0.9879921259842519

Validation / Subject : 7 / Timepoint : 5 / Loss : 0.1974443793296814 / AP : 0.9903280633412573 / F1 : 0.9882633777600954 / AUROC : 0.988399528116398

Validation / Subject : 7 / Timepoint : 6 / Loss : 0.2007

Exception: Early Stopping

This cell is the training and validation loop for the 80th Quantile Dataset experiment.\
The model will be saved in "checkpoint9.pth" file.

In [ ]:
# 80th Quantile Dataset Experiment
torch.autograd.set_detect_anomaly(False)
if os.path.isfile('checkpoint9.pth'):
        print('resuming checkpoint experiment')
        checkpoint = torch.load('checkpoint9.pth', map_location='cuda')
else:
  checkpoint = {
    'epoch': 0,
    'subject': 0,
    'timeIdx':0,
    'encoder': None,
    'decoder': None,
    'optimizerE': None,
    'lossSubject': None,
    'prevLoss' : np.Inf,
    'patience': 0,
    'lossSubjectV': np.Inf,
    'lr': 1e-4
    }

# Argv setting
epc=10
sampleStrategy='uniform'
negSampleStrategy='inductive'
N=116 # Number of ROIs
numEdges=math.ceil(116*116*0.4)
nodeDim=N # One-Hot encoding as a node embedding
edgeDim=4 # Arbitrarily set
timeDim=100 # Half of the value mentioned in HOT encoder paper
channelDim=50 # The value mentioned in HOT encoder paper
latentDim=N # Positive node embedding and negative node embedding concatenated across dim=1
numFilter=4 # Following Triadic Decoder paper
B=10000
prob=0.7398
nodeFeat=torch.cat((torch.eye(N),torch.zeros((1,N))),dim=0).numpy()
edgeFeat=torch.zeros((numEdges+1,edgeDim)).numpy()
learning_rateE=1e-4
pt=2
patience=0
prevLoss=np.inf
lossSubjectV=np.inf


encoder=HOT(nodeFeat, edgeFeat, None, timeDim, channelDim, patch_size=8, num_state_vectors=32, num_layers=2, num2hop=0, dropout=0.1, max_input_sequence_length=4096, device='cuda')
encoder.apply(init_kaiming_normal)
decoder=TriadicDecoder(latentDim, numFilter, device='cuda')
decoder.apply(init_kaiming_normal)
model=nn.Sequential(encoder, decoder).to('cuda')
lossFunc=nn.BCELoss().to('cuda')
optimizerE=torch.optim.Adam(model.parameters(), lr=checkpoint['lr'])


for epoch in range(checkpoint['epoch'], epc):
  print("Epoch : "+str(epoch))
  if os.path.isfile('checkpoint9.pth'):
    checkpoint=torch.load('checkpoint9.pth', map_location='cuda')
  for i in range(checkpoint['subject'], 16):
    if os.path.isfile('checkpoint9.pth'):
      checkpoint=torch.load('checkpoint9.pth', map_location='cuda')
    model.train()
    if checkpoint['lr']!=0: learning_rateE=checkpoint['lr']
    if checkpoint['optimizerE'] is not None: optimizerE.load_state_dict(checkpoint['optimizerE'])
    if checkpoint['prevLoss'] is not None: prevLoss=checkpoint['prevLoss']
    if checkpoint['patience'] != 0: patience=checkpoint['patience']
    if checkpoint['lossSubjectV'] is not None: lossSubjectV=checkpoint['lossSubjectV']
    if checkpoint['encoder'] is not None: model[0].load_state_dict(checkpoint['encoder'])
    if checkpoint['decoder'] is not None: model[1].load_state_dict(checkpoint['decoder'])
    print("\nSubject index : "+str(i)+" out of 16")

    train3D=torch.load(os.path.join('DatasetSplit','train40'+str(i)+'.pth'))
    train3D=train3D.reshape(-1,3).detach().to('cpu').numpy()
    train3D=train3D.reshape(15, -1, 3)

    print("Sampling probability : "+str(prob))

    lossSubject=torch.zeros(1).to('cuda')
    lossSubject.requires_grad_()

    for t in range(checkpoint['timeIdx'], 15):
      print("Time index : "+str(t)+" out of 15")

      timeBatch=train3D[t]
      srcNodes=timeBatch[:,0]
      dstNodes=timeBatch[:,1]
      timepoints=timeBatch[:,2]

      adjMat=torch.zeros((N,N)).to('cuda')
      for row in timeBatch:
        adjMat[int(row[0]), int(row[1])]=1

      nodePosEmbedding=torch.zeros((N,N)).to('cuda')
      nodeNegEmbedding=torch.zeros((N,N)).to('cuda')
      nodePosTimes=torch.zeros(N).to('cuda')
      nodeNegTimes=torch.zeros(N).to('cuda')
      nodeEmbedding=torch.zeros((N,N)).to('cuda')

      # Sample neighbor edges among the dynamic graphs of a single subject
      train_neighbor_sampler = get_neighbor_sampler(data=timeBatch, sample_neighbor_strategy=sampleStrategy, seed=0)
      # Sample negative edges among the dynamic graphs of a single subject
      train_neg_edge_sampler = NegativeEdgeSampler(src_node_ids=srcNodes, dst_node_ids=dstNodes, interact_times=timepoints, last_observed_time=np.min(timepoints), negative_sample_strategy=negSampleStrategy, seed=1)
      model[0].set_neighbor_sampler(train_neighbor_sampler)

      subject_src_node_embeddings, subject_dst_node_embeddings = \
        torch.utils.checkpoint.checkpoint(model[0].compute_src_dst_node_temporal_embeddings, srcNodes, dstNodes, timepoints, use_reentrant=False)
      negSrcNodes, negDstNodes = train_neg_edge_sampler.sample(len(srcNodes), srcNodes, dstNodes, np.min(timepoints), np.max(timepoints))
      subject_neg_src_node_embeddings, subject_neg_dst_node_embeddings = \
        torch.utils.checkpoint.checkpoint(model[0].compute_src_dst_node_temporal_embeddings, negSrcNodes, negDstNodes, timepoints, use_reentrant=False)

      for tb, row in enumerate(zip(srcNodes, dstNodes)):
        nodePosEmbedding[row[0]]=nodePosEmbedding[row[0]]+subject_src_node_embeddings[tb]
        nodePosTimes[row[0]]+=1
        nodePosEmbedding[row[1]]=nodePosEmbedding[row[1]]+subject_dst_node_embeddings[tb]
        nodePosTimes[row[1]]+=1
      for tb, row in enumerate(zip(negSrcNodes, negDstNodes)):
        nodeNegEmbedding[row[0]]=nodeNegEmbedding[row[0]]+subject_neg_src_node_embeddings[tb]
        nodeNegTimes[row[0]]+=1
        nodeNegEmbedding[row[1]]=nodeNegEmbedding[row[1]]+subject_neg_dst_node_embeddings[tb]
        nodeNegTimes[row[1]]+=1

      for node in range(N):
        if nodePosTimes[node]>0:
          nodePosEmbedding[node]=nodePosEmbedding[node]/nodePosTimes[node]
        if nodeNegTimes[node]>0:
          nodeNegEmbedding[node]=nodeNegEmbedding[node]/nodeNegTimes[node]


      nodePosEmbedding=(nodePosEmbedding-torch.mean(nodePosEmbedding, dim=0, keepdims=True))/(torch.std(nodePosEmbedding, dim=0, keepdims=True)+1e-10)
      nodeNegEmbedding=(nodeNegEmbedding-torch.mean(nodeNegEmbedding, dim=0, keepdims=True))/(torch.std(nodeNegEmbedding, dim=0, keepdims=True)+1e-10)

      graph2D=np.concatenate((np.expand_dims(srcNodes, axis=1), np.expand_dims(dstNodes, axis=1)), axis=1)
      graph2D=torch.tensor(graph2D).to('cuda')
      graph2DN=np.concatenate((np.expand_dims(negSrcNodes, axis=1), np.expand_dims(negDstNodes, axis=1)), axis=1)
      graph2DN=torch.tensor(graph2DN).to('cuda')
      triads=triadSample(graph2D, B, prob, device='cuda')

      mSquare=torch.zeros((N,N)).to('cuda')
      eSquare=torch.zeros((N,N)).to('cuda')

      for j, triad in enumerate(triads):
        triad0=triad[0].unsqueeze(0).unsqueeze(0)
        triad1=triad[1].unsqueeze(0).unsqueeze(0)
        triad2=triad[2].unsqueeze(0).unsqueeze(0)

        if mSquare[triad[0], triad[1]]==0:
          pair1=torch.cat((triad0, triad1), dim=1).to('cuda')
          M1=calculateM(triads, pair1)
          mSquare[triad[0], triad[1]]=M1
        if mSquare[triad[0], triad[2]]==0:
          pair2=torch.cat((triad0, triad2), dim=1).to('cuda')
          M2=calculateM(triads, pair2)
          mSquare[triad[0], triad[2]]=M2
        if mSquare[triad[1], triad[2]]==0:
          pair3=torch.cat((triad1, triad2), dim=1).to('cuda')
          M3=calculateM(triads, pair3)
          mSquare[triad[1], triad[2]]=M3

      for j, triad in enumerate(triads):
        triad0=triad[0].unsqueeze(0).unsqueeze(0)
        triad1=triad[1].unsqueeze(0).unsqueeze(0)
        triad2=triad[2].unsqueeze(0).unsqueeze(0)
        pair1=torch.cat((triad0, triad1), dim=1).squeeze(0).to('cuda')
        pair2=torch.cat((triad0, triad2), dim=1).squeeze(0).to('cuda')
        pair3=torch.cat((triad1, triad2), dim=1).squeeze(0).to('cuda')

        emb1=torch.zeros((1,N)).to('cuda')
        emb2=torch.zeros((1,N)).to('cuda')
        emb3=torch.zeros((1,N)).to('cuda')

        if pairIn(pair1,graph2D[:,0:2]):
          emb1=nodePosEmbedding[triad[0]]
        if pairIn(pair1,graph2DN):
          emb1=nodeNegEmbedding[triad[0]]
        if pairIn(pair2,graph2D[:,0:2]):
          emb2=nodePosEmbedding[triad[1]]
        if pairIn(pair2,graph2DN):
          emb2=nodeNegEmbedding[triad[1]]
        if pairIn(pair3,graph2D[:,0:2]):
          emb3=nodePosEmbedding[triad[2]]
        if pairIn(pair3,graph2DN):
          emb3=nodeNegEmbedding[triad[2]]

        eTriplet=model[1].forward(emb1, emb2, emb3)
        if pairIn(pair1,graph2D[:,0:2]):
          eSquare[triad[0], triad[1]]+=eTriplet[0]
        if pairIn(pair1,graph2DN):
          eSquare[triad[0], triad[1]]-=eTriplet[0]
        if pairIn(pair2,graph2D[:,0:2]):
          eSquare[triad[0], triad[2]]+=eTriplet[1]
        if pairIn(pair2,graph2DN):
          eSquare[triad[0], triad[2]]-=eTriplet[1]
        if pairIn(pair3,graph2D[:,0:2]):
          eSquare[triad[1], triad[2]]+=eTriplet[2]
        if pairIn(pair3,graph2DN):
          eSquare[triad[1], triad[2]]-=eTriplet[2]
      eSquare.clamp_(min=0.0)

      nonzeros=((mSquare>0).nonzero())
      adjRecon=[]
      adjPart=[]
      adjFull=torch.zeros((N,N)).to('cuda')
      for nz in nonzeros:
        nzPair=torch.tensor([int(nz[0]), int(nz[1])]).to('cuda')
        em1=eSquare[int(nz[0]), int(nz[1])]/mSquare[int(nz[0]), int(nz[1])]
        if pairIn(nzPair,graph2D[:,0:2]):
          adjRecon.append(em1)
          adjPart.append(torch.tensor([1.0]))
        if pairIn(nzPair, graph2DN):
          adjRecon.append(em1)
          adjPart.append(torch.tensor([0.0]))
        if em1>0.5:
          adjFull[int(nz[0]), int(nz[1])]=1
        else:
          adjFull[int(nz[0]), int(nz[1])]=0
      adjRecon=torch.stack(adjRecon).to('cuda')
      print(torch.mean(adjRecon))
      adjPart=torch.stack(adjPart).squeeze(1).to('cuda')



      print(torch.nonzero(adjFull[40])[:,0])
      loss=lossFunc(input=adjRecon, target=adjPart)
      lossSubject=lossSubject+loss

      adjReconC=adjRecon.clone().detach().cpu().numpy()
      adjPartC=adjPart.clone().detach().cpu().numpy()


      adjReconC[adjReconC>0.5]=1
      adjReconC[adjReconC<=0.5]=0
      F1=f1_score(adjPartC, adjReconC)
      AUROC=roc_auc_score(adjPartC, adjReconC)
      AP=average_precision_score(adjPartC, adjReconC)


      print("Epoch : "+str(epoch)+" / Subject : "+str(i)+" / Timepoint : "+str(t) +" / Loss : "+str(loss.item())+" / AUROC : "+str(AUROC)+" / F1 : "+str(F1)+" / AP : "+str(AP)+"\n")

      torch.cuda.empty_cache()

    optimizerE.zero_grad()
    lossSubject.backward()
    optimizerE.step()

    # Validation Step
    if (i+1)%2==0:
      model.eval()
      with torch.no_grad():
        patience=checkpoint['patience']
        prevLoss=checkpoint['prevLoss']
        valid3D=torch.load(os.path.join('DatasetSplit','valid40'+str((i+1)//2-1)+'.pth'))
        valid3D=valid3D.reshape(-1, 3).detach().to('cpu').numpy()
        valid3D=valid3D.reshape(15, -1, 3)
        lossSubjectV=0

        print("Validation Subject index : "+str((i+1)//2-1)+" out of 8")
        for t in range(15):
          adjRecon=torch.zeros((N,N)).to('cuda')

          timeBatch=valid3D[t]
          srcNodes=timeBatch[:,0]
          dstNodes=timeBatch[:,1]
          timepoints=timeBatch[:,2]

          adjMat=torch.zeros((N,N)).to('cuda')
          for row in timeBatch:
            adjMat[int(row[0]), int(row[1])]=1

          nodePosEmbedding=torch.zeros((N,N)).to('cuda')
          nodeNegEmbedding=torch.zeros((N,N)).to('cuda')
          nodePosTimes=torch.zeros(N).to('cuda')
          nodeNegTimes=torch.zeros(N).to('cuda')
          nodeEmbedding=torch.zeros((N,N)).to('cuda')

          # Sample neighbor edges among the dynamic graphs of a single subject
          valid_neighbor_sampler = get_neighbor_sampler(data=timeBatch, sample_neighbor_strategy=sampleStrategy, seed=2)
          # Sample negative edges among the dynamic graphs of a single subject
          valid_neg_edge_sampler = NegativeEdgeSampler(src_node_ids=srcNodes, dst_node_ids=dstNodes, interact_times=timepoints, last_observed_time=np.min(timepoints), negative_sample_strategy=negSampleStrategy, seed=3)
          model[0].set_neighbor_sampler(valid_neighbor_sampler)

          subject_src_node_embeddings, subject_dst_node_embeddings = \
            model[0].compute_src_dst_node_temporal_embeddings(src_node_ids=srcNodes, dst_node_ids=dstNodes, node_interact_times=timepoints)
          negSrcNodes, negDstNodes = valid_neg_edge_sampler.sample(len(srcNodes), srcNodes, dstNodes, np.min(timepoints), np.max(timepoints))
          subject_neg_src_node_embeddings, subject_neg_dst_node_embeddings = \
            model[0].compute_src_dst_node_temporal_embeddings(src_node_ids=negSrcNodes,
                                                                          dst_node_ids=negDstNodes,
                                                                          node_interact_times=timepoints)

          for tb, row in enumerate(zip(srcNodes, dstNodes)):
            nodePosEmbedding[row[0]]=nodePosEmbedding[row[0]]+subject_src_node_embeddings[tb]
            nodePosTimes[row[0]]+=1
            nodePosEmbedding[row[1]]=nodePosEmbedding[row[1]]+subject_dst_node_embeddings[tb]
            nodePosTimes[row[1]]+=1
          for tb, row in enumerate(zip(negSrcNodes, negDstNodes)):
            nodeNegEmbedding[row[0]]=nodeNegEmbedding[row[0]]+subject_neg_src_node_embeddings[tb]
            nodeNegTimes[row[0]]+=1
            nodeNegEmbedding[row[1]]=nodeNegEmbedding[row[1]]+subject_neg_dst_node_embeddings[tb]
            nodeNegTimes[row[1]]+=1

          for node in range(N):
            if nodePosTimes[node]>0:
              nodePosEmbedding[node]=nodePosEmbedding[node]/nodePosTimes[node]
            if nodeNegTimes[node]>0:
              nodeNegEmbedding[node]=nodeNegEmbedding[node]/nodeNegTimes[node]

          nodePosEmbedding=(nodePosEmbedding-torch.mean(nodePosEmbedding, dim=0, keepdims=True))/(torch.std(nodePosEmbedding, dim=0, keepdims=True)+1e-10)
          nodeNegEmbedding=(nodeNegEmbedding-torch.mean(nodeNegEmbedding, dim=0, keepdims=True))/(torch.std(nodeNegEmbedding, dim=0, keepdims=True)+1e-10)

          graph2D=np.concatenate((np.expand_dims(srcNodes, axis=1), np.expand_dims(dstNodes, axis=1)), axis=1)
          graph2D=torch.tensor(graph2D).to('cuda')
          graph2DN=np.concatenate((np.expand_dims(negSrcNodes, axis=1), np.expand_dims(negDstNodes, axis=1)), axis=1)
          graph2DN=torch.tensor(graph2DN).to('cuda')

          triads=triadSample(graph2D, B, prob, device='cuda')
          mSquare=torch.zeros((N,N)).to('cuda')
          eSquare=torch.zeros((N,N)).to('cuda')

          for j, triad in enumerate(triads):
            triad0=triad[0].unsqueeze(0).unsqueeze(0)
            triad1=triad[1].unsqueeze(0).unsqueeze(0)
            triad2=triad[2].unsqueeze(0).unsqueeze(0)

            if mSquare[triad[0], triad[1]]==0:
              pair1=torch.cat((triad0, triad1), dim=1).to('cuda')
              M1=calculateM(triads, pair1)
              mSquare[triad[0], triad[1]]=M1

            if mSquare[triad[0], triad[2]]==0:
              pair1=torch.cat((triad0, triad2), dim=1).to('cuda')
              M2=calculateM(triads, pair1)
              mSquare[triad[0], triad[2]]=M2

            if mSquare[triad[1], triad[2]]==0:
              pair1=torch.cat((triad1, triad2), dim=1).to('cuda')
              M3=calculateM(triads, pair1)
              mSquare[triad[1], triad[2]]=M3

          for j, triad in enumerate(triads):
            triad0=triad[0].unsqueeze(0).unsqueeze(0)
            triad1=triad[1].unsqueeze(0).unsqueeze(0)
            triad2=triad[2].unsqueeze(0).unsqueeze(0)
            pair1=torch.cat((triad0, triad1), dim=1).squeeze(0).to('cuda')
            pair2=torch.cat((triad0, triad2), dim=1).squeeze(0).to('cuda')
            pair3=torch.cat((triad1, triad2), dim=1).squeeze(0).to('cuda')

            if pair1 in graph2D[:,0:2]:
              emb1=nodePosEmbedding[triad[0]]
            if pair1 in graph2DN:
              emb1=nodeNegEmbedding[triad[0]]
            if pair2 in graph2D[:,0:2]:
              emb2=nodePosEmbedding[triad[1]]
            if pair2 in graph2DN:
              emb2=nodeNegEmbedding[triad[1]]
            if pair3 in graph2D[:,0:2]:
              emb3=nodePosEmbedding[triad[2]]
            if pair3 in graph2DN:
              emb3=nodeNegEmbedding[triad[2]]

            eTriplet=model[1].forward(emb1, emb2, emb3)
            if pairIn(pair1,graph2D[:,0:2]):
              eSquare[triad[0], triad[1]]+=eTriplet[0]
            if pairIn(pair1,graph2DN):
              eSquare[triad[0], triad[1]]-=eTriplet[0]
            if pairIn(pair2,graph2D[:,0:2]):
              eSquare[triad[0], triad[2]]+=eTriplet[1]
            if pairIn(pair2,graph2DN):
              eSquare[triad[0], triad[2]]-=eTriplet[1]
            if pairIn(pair3,graph2D[:,0:2]):
              eSquare[triad[1], triad[2]]+=eTriplet[2]
            if pairIn(pair3,graph2DN):
              eSquare[triad[1], triad[2]]-=eTriplet[2]

          eSquare.clamp_(min=0.0)
          nonzeros=((mSquare>0).nonzero())
          adjRecon=[]
          adjPart=[]
          adjFull=torch.zeros((N,N)).to('cuda')
          for nz in nonzeros:
            nzPair=torch.tensor([int(nz[0]), int(nz[1])]).to('cuda')
            em1=eSquare[int(nz[0]), int(nz[1])]/mSquare[int(nz[0]), int(nz[1])]
            if pairIn(nzPair, graph2D[:,0:2]):
              adjRecon.append(em1)
              adjPart.append(torch.tensor([1.0]))
            if pairIn(nzPair,graph2DN):
              adjRecon.append(em1)
              adjPart.append(torch.tensor([0.0]))
            if em1>0.5:
              adjFull[int(nz[0]), int(nz[1])]=1
            else:
              adjFull[int(nz[0]), int(nz[1])]=0
          adjRecon=torch.stack(adjRecon).to('cuda')
          adjPart=torch.stack(adjPart).squeeze(1).to('cuda')


          loss=lossFunc(input=adjRecon, target=adjPart)
          lossSubjectV=lossSubjectV+loss

          adjReconC=adjRecon.clone().detach().cpu().numpy()
          adjPartC=adjPart.clone().detach().cpu().numpy()

          adjReconC[adjReconC>0.5]=1
          adjReconC[adjReconC<=0.5]=0
          F1=f1_score(adjPartC, adjReconC)
          AUROC=roc_auc_score(adjPartC, adjReconC)
          AP=average_precision_score(adjPartC, adjReconC)

          print("Validation / Subject : "+str((i+1)//2-1)+" / Timepoint : "+str(t) +" / Loss : "+str(loss.item())+" / AP : "+str(AP)+" / F1 : "+str(F1)+" / AUROC : "+str(AUROC)+'\n')


        if lossSubjectV>prevLoss:
          patience+=1
        else:
          patience=0

        if patience==1 and learning_rateE<=2.5e-5:
          torch.save({
          'epoch': epoch,
          'encoder': model[0].state_dict(),
          'decoder': model[1].state_dict(),
          'subject': i,
          'timeIdx': t,
          'optimizerE': optimizerE.state_dict(),
          'lossSubject': lossSubject,
          'patience': patience,
          'prevLoss': prevLoss,
          'lossSubjectV': lossSubjectV,
          'lr': learning_rateE
          },
          'checkpoint9.pth')
          raise Exception('Early Stopping')
        if patience==pt and learning_rateE>2.5e-5:
          learning_rateE=learning_rateE/2
          patience=0

    if lossSubjectV is not None:
      prevLoss=lossSubjectV
    torch.save({
        'epoch': epoch,
        'encoder': model[0].state_dict(),
        'decoder': model[1].state_dict(),
        'subject': i+1,
        'timeIdx': 0,
        'optimizerE': optimizerE.state_dict(),
        'lossSubject': None,
        'patience': patience,
        'prevLoss': prevLoss,
        'lossSubjectV': lossSubjectV,
        'lr': learning_rateE
        },
        'checkpoint9.pth')

    torch.cuda.empty_cache()

  torch.save({
              'epoch': epoch+1,
              'encoder': model[0].state_dict(),
              'decoder': model[1].state_dict(),
              'subject': 0,
              'timeIdx': 0,
              'optimizerE': optimizerE.state_dict(),
              'lossSubject': None,
              'patience': patience,
              'prevLoss': prevLoss,
              'lossSubjectV': lossSubjectV,
              'lr': learning_rateE
              },
              'checkpoint9.pth')


resuming checkpoint experiment


<ipython-input-8-e2e187f6af67>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint9.pth', map_location='cuda')


Epoch : 0


<ipython-input-8-e2e187f6af67>:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load('checkpoint9.pth', map_location='cuda')
<ipython-input-8-e2e187f6af67>


Subject index : 9 out of 16


...

<ipython-input-8-e2e187f6af67>:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train3D=torch.load(os.path.join('DatasetSplit','train40'+str(i)+'.pth'))


tensor(0.3923, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([0, 1], device='cuda:0')
Epoch : 2 / Subject : 3 / Timepoint : 0 / Loss : 0.243011936545372 / AUROC : 0.9557737169517885 / F1 : 0.9537272449913556 / AP : 0.9630821490468549

Time index : 1 out of 15
tensor(0.3932, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Epoch : 2 / Subject : 3 / Timepoint : 1 / Loss : 0.24735811352729797 / AUROC : 0.9519818576217709 / F1 : 0.9495598135680994 / AP : 0.9602317683920107

Time index : 2 out of 15
tensor(0.3953, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 14, 20, 26, 40], device='cuda:0')
Epoch : 2 / Subject : 3 / Timepoint : 2 / Loss : 0.24320609867572784 / AUROC : 0.9531436530043773 / F1 : 0.9508402045715478 / AP : 0.9611568307058654

Time index : 3 out of 15
tensor(0.3938, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([15, 26, 40], device='cuda:0')
Epoch : 2 / Subject : 3 / Timepoint : 3 / Loss : 0.24458646774291992 / AUROC : 0.9533678

<ipython-input-8-e2e187f6af67>:254: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  valid3D=torch.load(os.path.join('DatasetSplit','valid40'+str((i+1)//2-1)+'.pth'))


Validation Subject index : 1 out of 8
Validation / Subject : 1 / Timepoint : 0 / Loss : 0.20948141813278198 / AP : 0.9773184077752715 / F1 : 0.9729946792490713 / AUROC : 0.973704789833822

Validation / Subject : 1 / Timepoint : 1 / Loss : 0.21194475889205933 / AP : 0.9768780307980238 / F1 : 0.972152407761134 / AUROC : 0.9729068857589984

Validation / Subject : 1 / Timepoint : 2 / Loss : 0.2131851315498352 / AP : 0.9749184150878355 / F1 : 0.9699157641395909 / AUROC : 0.9707943925233644

Validation / Subject : 1 / Timepoint : 3 / Loss : 0.22286032140254974 / AP : 0.9692961979331771 / F1 : 0.9625290374709625 / AUROC : 0.963882398753894

Validation / Subject : 1 / Timepoint : 4 / Loss : 0.22532296180725098 / AP : 0.9674873597866345 / F1 : 0.9601778855872246 / AUROC : 0.9617029548989113

Validation / Subject : 1 / Timepoint : 5 / Loss : 0.22091566026210785 / AP : 0.9692566950278158 / F1 : 0.9628128483128989 / AUROC : 0.964146150840172

Validation / Subject : 1 / Timepoint : 6 / Loss : 0.219

Exception: Early Stopping

The cell below installs and initializes the wandb session, which is a dashboard service to record experimental details and compare the results.

In [ ]:
# Installing wandb
!pip install wandb
import wandb
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


The two cells below is for testing the model trained with the 40th Quantile Dataset.

In [ ]:
# 40th Quantile Dataset Testing

sampleStrategy='uniform'
negSampleStrategy='inductive'
N=116 # Number of ROIs
numEdges=math.ceil(116*116*0.4)
nodeDim=N # One-Hot encoding as a node embedding
edgeDim=4 # Arbitrarily set
timeDim=100 # Half of the value mentioned in D=the HOT paper
channelDim=50 # Half of the value mentioned in the HOT paper
latentDim=N # Positive node embedding and negative node embedding concatenated across dim=1
numFilter=4 # Following Triadic Decoder paper
B=10000
prob=0.7398
nodeFeat=torch.cat((torch.eye(N),torch.zeros((1,N))),dim=0).numpy()
edgeFeat=torch.zeros((numEdges+1,edgeDim)).numpy()

wandb.init(
    project="EmotionConnectivity - 40thQuantileTest",
    config={
        "Mode" : "Test",
        "initialization" : "kaiming_normal",
        "sampleStrategy" : sampleStrategy,
        "negSampleStrategy" : negSampleStrategy,
        "B": B,
        "prob": prob,
        "Dropout" : 0.1,
    }
)

checkpoint=torch.load('checkpoint8.pth')
encoder=HOT(nodeFeat, edgeFeat, None, timeDim, channelDim, patch_size=8, num_state_vectors=32, num_layers=2, num2hop=0, dropout=0.1, max_input_sequence_length=4096, device='cuda')
encoder.load_state_dict(checkpoint['encoder'])
encoder.eval()
decoder=TriadicDecoder(latentDim, numFilter, device='cuda')
decoder.load_state_dict(checkpoint['decoder'])
decoder.eval()
model=nn.Sequential(encoder,  decoder).to('cuda')
lossFunc=nn.BCELoss()
data=datasets.fetch_atlas_aal()
connDictAmygL={}
connDictInsL={}

with torch.no_grad():
  for i in range(10):
    model.eval()
    print("\nSubject index : "+str(i)+" out of 10")
    test3D=torch.load(os.path.join('DatasetSplit','test'+str(i)+'.pth'))
    test3D=test3D.reshape(-1,3).detach().to('cpu').numpy()
    test3D=test3D.reshape(15, -1, 3)
    print("Sampling probability : "+str(prob))

    lossSubject=0
    F1Subject=0
    APSubject=0
    AUROCSubject=0
    for t in range(15):

      print("Time index : "+str(t)+" out of 15")

      timeBatch=test3D[t]
      srcNodes=timeBatch[:,0]
      dstNodes=timeBatch[:,1]
      timepoints=timeBatch[:,2]

      adjMat=torch.zeros((N,N)).to('cuda')
      for row in timeBatch:
        adjMat[int(row[0]), int(row[1])]=1

      nodePosEmbedding=torch.zeros((N,N)).to('cuda')
      nodeNegEmbedding=torch.zeros((N,N)).to('cuda')
      nodePosTimes=torch.zeros(N).to('cuda')
      nodeNegTimes=torch.zeros(N).to('cuda')
      nodeEmbedding=torch.zeros((N,N)).to('cuda')

      test_neighbor_sampler = get_neighbor_sampler(data=timeBatch, sample_neighbor_strategy=sampleStrategy, seed=4)
      test_neg_edge_sampler = NegativeEdgeSampler(src_node_ids=srcNodes, dst_node_ids=dstNodes, interact_times=timepoints, last_observed_time=np.min(timepoints), negative_sample_strategy=negSampleStrategy, seed=5)
      model[0].set_neighbor_sampler(test_neighbor_sampler)

      subject_src_node_embeddings, subject_dst_node_embeddings = \
        model[0].compute_src_dst_node_temporal_embeddings(src_node_ids=srcNodes, dst_node_ids=dstNodes, node_interact_times=timepoints)
      negSrcNodes, negDstNodes = test_neg_edge_sampler.sample(len(srcNodes), srcNodes, dstNodes, np.min(timepoints), np.max(timepoints))
      subject_neg_src_node_embeddings, subject_neg_dst_node_embeddings = \
        model[0].compute_src_dst_node_temporal_embeddings(src_node_ids=negSrcNodes,
                                                                      dst_node_ids=negDstNodes,
                                                                      node_interact_times=timepoints)

      for tb, row in enumerate(zip(srcNodes, dstNodes)):
        nodePosEmbedding[row[0]]=nodePosEmbedding[row[0]]+subject_src_node_embeddings[tb]
        nodePosTimes[row[0]]+=1
        nodePosEmbedding[row[1]]=nodePosEmbedding[row[1]]+subject_dst_node_embeddings[tb]
        nodePosTimes[row[1]]+=1
      for tb, row in enumerate(zip(negSrcNodes, negDstNodes)):
        nodeNegEmbedding[row[0]]=nodeNegEmbedding[row[0]]+subject_neg_src_node_embeddings[tb]
        nodeNegTimes[row[0]]+=1
        nodeNegEmbedding[row[1]]=nodeNegEmbedding[row[1]]+subject_neg_dst_node_embeddings[tb]
        nodeNegTimes[row[1]]+=1

      for node in range(N):
        if nodePosTimes[node]>0:
          nodePosEmbedding[node]=nodePosEmbedding[node]/nodePosTimes[node]
        if nodeNegTimes[node]>0:
          nodeNegEmbedding[node]=nodeNegEmbedding[node]/nodeNegTimes[node]

      nodePosEmbedding=(nodePosEmbedding-torch.mean(nodePosEmbedding, dim=0, keepdims=True))/(torch.std(nodePosEmbedding, dim=0, keepdims=True)+1e-10)
      nodeNegEmbedding=(nodeNegEmbedding-torch.mean(nodeNegEmbedding, dim=0, keepdims=True))/(torch.std(nodeNegEmbedding, dim=0, keepdims=True)+1e-10)

      graph2D=np.concatenate((np.expand_dims(srcNodes, axis=1), np.expand_dims(dstNodes, axis=1)), axis=1)
      graph2D=torch.tensor(graph2D).to('cuda')
      graph2DN=np.concatenate((np.expand_dims(negSrcNodes, axis=1), np.expand_dims(negDstNodes, axis=1)), axis=1)
      graph2DN=torch.tensor(graph2DN).to('cuda')

      triads=triadSample(graph2D, B, prob, device='cuda')
      mSquare=torch.zeros((N,N)).to('cuda')
      eSquare=torch.zeros((N,N)).to('cuda')

      for j, triad in enumerate(triads):
        triad0=triad[0].unsqueeze(0).unsqueeze(0)
        triad1=triad[1].unsqueeze(0).unsqueeze(0)
        triad2=triad[2].unsqueeze(0).unsqueeze(0)

        if mSquare[triad[0], triad[1]]==0:
          pair1=torch.cat((triad0, triad1), dim=1).to('cuda')
          M1=calculateM(triads, pair1)
          mSquare[triad[0], triad[1]]=M1

        if mSquare[triad[0], triad[2]]==0:
          pair1=torch.cat((triad0, triad2), dim=1).to('cuda')
          M2=calculateM(triads, pair1)
          mSquare[triad[0], triad[2]]=M2

        if mSquare[triad[1], triad[2]]==0:
          pair1=torch.cat((triad1, triad2), dim=1).to('cuda')
          M3=calculateM(triads, pair1)
          mSquare[triad[1], triad[2]]=M3

      for j, triad in enumerate(triads):
        triad0=triad[0].unsqueeze(0).unsqueeze(0)
        triad1=triad[1].unsqueeze(0).unsqueeze(0)
        triad2=triad[2].unsqueeze(0).unsqueeze(0)
        pair1=torch.cat((triad0, triad1), dim=1).squeeze(0).to('cuda')
        pair2=torch.cat((triad0, triad2), dim=1).squeeze(0).to('cuda')
        pair3=torch.cat((triad1, triad2), dim=1).squeeze(0).to('cuda')

        if pair1 in graph2D[:,0:2]:
          emb1=nodePosEmbedding[triad[0]]
        if pair1 in graph2DN:
          emb1=nodeNegEmbedding[triad[0]]
        if pair2 in graph2D[:,0:2]:
          emb2=nodePosEmbedding[triad[1]]
        if pair2 in graph2DN:
          emb2=nodeNegEmbedding[triad[1]]
        if pair3 in graph2D[:,0:2]:
          emb3=nodePosEmbedding[triad[2]]
        if pair3 in graph2DN:
          emb3=nodeNegEmbedding[triad[2]]

        eTriplet=model[1].forward(emb1, emb2, emb3)
        if pairIn(pair1,graph2D[:,0:2]):
          eSquare[triad[0], triad[1]]+=eTriplet[0]
        if pairIn(pair1,graph2DN):
          eSquare[triad[0], triad[1]]-=eTriplet[0]
        if pairIn(pair2,graph2D[:,0:2]):
          eSquare[triad[0], triad[2]]+=eTriplet[1]
        if pairIn(pair2,graph2DN):
          eSquare[triad[0], triad[2]]-=eTriplet[1]
        if pairIn(pair3,graph2D[:,0:2]):
          eSquare[triad[1], triad[2]]+=eTriplet[2]
        if pairIn(pair3,graph2DN):
          eSquare[triad[1], triad[2]]-=eTriplet[2]

      eSquare.clamp_(min=0.0)
      nonzeros=((mSquare>0).nonzero())
      adjRecon=[]
      adjPart=[]
      adjFull=torch.zeros((N,N)).to('cuda')
      for nz in nonzeros:
        nzPair=torch.tensor([int(nz[0]), int(nz[1])]).to('cuda')
        em1=eSquare[int(nz[0]), int(nz[1])]/mSquare[int(nz[0]), int(nz[1])]
        if pairIn(nzPair, graph2D[:,0:2]):
          adjRecon.append(em1)
          adjPart.append(torch.tensor([1.0]))
        if pairIn(nzPair,graph2DN):
          adjRecon.append(em1)
          adjPart.append(torch.tensor([0.0]))
        if em1>0.5:
          adjFull[int(nz[0]), int(nz[1])]=1
        else:
          adjFull[int(nz[0]), int(nz[1])]=0
      adjRecon=torch.stack(adjRecon).to('cuda')
      adjPart=torch.stack(adjPart).squeeze(1).to('cuda')

      loss=lossFunc(input=adjRecon, target=adjPart)
      lossSubject+=loss
      adjReconC=adjRecon.clone().detach().cpu().numpy()
      adjPartC=adjPart.clone().detach().cpu().numpy()

      adjReconC[adjReconC>0.5]=1
      adjReconC[adjReconC<=0.5]=0
      F1=f1_score(adjPartC, adjReconC)
      AUROC=roc_auc_score(adjPartC, adjReconC)
      AP=average_precision_score(adjPartC, adjReconC)
      F1Subject+=F1
      APSubject+=AP
      AUROCSubject+=AUROC

      print("Test / Subject : "+str(i)+" / Timepoint : "+str(t) +" / Loss : "+str(loss.item())+" / AUROC : "+str(AUROC)+" / F1 : "+str(F1)+" / AP : "+str(AP)+"\n")


      AmygLConn=torch.nonzero(adjFull[40])[:,0]
      for elem in AmygLConn:
        if data.labels[elem] not in connDictAmygL.keys():
          connDictAmygL[data.labels[elem]]=1
        else:
          connDictAmygL[data.labels[elem]]+=1

      InsLConn=torch.nonzero(adjFull[28])[:,0]
      for elem in InsLConn:
        if data.labels[elem] not in connDictInsL.keys():
          connDictInsL[data.labels[elem]]=1
        else:
          connDictInsL[data.labels[elem]]+=1

    wandb.log({"TLoss": lossSubject.item()/15, "T_AP": APSubject/15, "T_F1": F1Subject/15, "T_AUROC":AUROCSubject/15})
    torch.cuda.empty_cache()

torch.save(connDictAmygL, 'connDictTestAmygL.pth')
torch.save(connDictInsL, 'connDictTestInsL.pth')

<ipython-input-18-1d96a1356a99>:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load('checkpoint8.pth')
<ipython-input-18-1d96a1356a99>:46: FutureWarning:


Subject index : 0 out of 10
Sampling probability : 0.7398
Time index : 0 out of 15
tensor([  8,   9,  14,  15,  19,  25,  31,  32,  33,  35,  37,  40,  54,  55,
         74,  82,  96,  97, 104, 108, 111, 115], device='cuda:0')
Test / Subject : 0 / Timepoint : 0 / Loss : 0.19278858602046967 / AUROC : 0.9902054926061072 / F1 : 0.9901086113266098 / AP : 0.9916804581986247

Time index : 1 out of 15
tensor([ 15,  19,  25,  32,  33,  35,  37,  40,  41,  48,  54,  82, 106, 111],
       device='cuda:0')
Test / Subject : 0 / Timepoint : 1 / Loss : 0.19345472753047943 / AUROC : 0.9889298892988929 / F1 : 0.9888059701492538 / AP : 0.9905546561033539

Time index : 2 out of 15
tensor([37, 40, 48, 82], device='cuda:0')
Test / Subject : 0 / Timepoint : 2 / Loss : 0.19252915680408478 / AUROC : 0.9891981315687037 / F1 : 0.9890801770782095 / AP : 0.9907337089526793

Time index : 3 out of 15
tensor([ 25,  40,  82, 107], device='cuda:0')
Test / Subject : 0 / Timepoint : 3 / Loss : 0.19231298565864563 / AU

<ipython-input-18-1d96a1356a99>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test3D=torch.load(os.path.join('DatasetSplit','test'+str(i)+'.pth'))


Sampling probability : 0.7398
Time index : 0 out of 15
tensor([ 19,  20,  21,  25,  26,  28,  30,  31,  35,  36,  40,  45,  47,  49,
         50,  58,  59,  60,  61,  66,  67,  70,  73,  75,  80,  83,  85,  90,
         92,  95,  96,  98,  99, 100, 102, 103, 104, 105, 109, 112, 114],
       device='cuda:0')
Test / Subject : 1 / Timepoint : 0 / Loss : 0.19138039648532867 / AUROC : 0.9900329011031546 / F1 : 0.9899325579122276 / AP : 0.9914382213296393

Time index : 1 out of 15
tensor([ 25,  31,  33,  36,  40,  42,  43,  47,  59,  60,  62,  66,  67,  75,
         83,  85,  90,  94,  95,  96, 100, 104, 109, 114], device='cuda:0')
Test / Subject : 1 / Timepoint : 1 / Loss : 0.19226908683776855 / AUROC : 0.9898629078972775 / F1 : 0.9897590948990539 / AP : 0.991335722915298

Time index : 2 out of 15
tensor([  1,  11,  28,  36,  40,  41,  47,  49,  50,  51,  53,  59,  60,  62,
         72,  75,  83,  85,  96, 114], device='cuda:0')
Test / Subject : 1 / Timepoint : 2 / Loss : 0.1934458464384079

<ipython-input-18-1d96a1356a99>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test3D=torch.load(os.path.join('DatasetSplit','test'+str(i)+'.pth'))


...

Sampling probability : 0.7398
Time index : 0 out of 15
tensor([  1,  16,  20,  21,  24,  25,  26,  33,  36,  38,  39,  40,  41,  42,
         43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  56,  57,
         59,  66,  67,  69,  70,  71,  73,  77,  80,  81,  82,  87,  89,  92,
         94,  95,  96,  97,  98,  99, 100, 110, 111, 112, 114],
       device='cuda:0')
Test / Subject : 7 / Timepoint : 0 / Loss : 0.19652020931243896 / AUROC : 0.987223823246878 / F1 : 0.9870584801011969 / AP : 0.989186121671061

Time index : 1 out of 15
tensor([  1,  16,  20,  21,  24,  25,  26,  34,  36,  37,  39,  40,  41,  42,
         43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  56,  57,
         59,  66,  67,  69,  70,  71,  73,  75,  87,  89,  94,  96,  97,  98,
        100, 110, 111, 114, 115], device='cuda:0')
Test / Subject : 7 / Timepoint : 1 / Loss : 0.19627973437309265 / AUROC : 0.9874304356169641 / F1 : 0.987270430473229 / AP : 0.9893584365283064

Time index : 2 out of 15

<ipython-input-18-1d96a1356a99>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test3D=torch.load(os.path.join('DatasetSplit','test'+str(i)+'.pth'))


Sampling probability : 0.7398
Time index : 0 out of 15
tensor([  1,   2,   3,   4,   5,   6,   7,   8,  10,  11,  12,  13,  16,  17,
         19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
         33,  37,  40,  41,  43,  45,  48,  49,  50,  51,  52,  55,  56,  58,
         59,  60,  63,  64,  67,  70,  71,  72,  73,  74,  76,  77,  78,  79,
         80,  81,  82,  83,  85,  86,  87,  88,  89,  95,  97, 113, 114],
       device='cuda:0')
Test / Subject : 8 / Timepoint : 0 / Loss : 0.1937376707792282 / AUROC : 0.989656518345043 / F1 : 0.989548412541905 / AP : 0.9912419135750735

Time index : 1 out of 15
tensor([  1,   3,   4,   5,   6,   7,   8,  11,  12,  13,  14,  16,  17,  18,
         19,  20,  21,  22,  23,  25,  26,  27,  28,  29,  30,  31,  32,  33,
         37,  40,  41,  43,  48,  49,  50,  52,  55,  57,  59,  62,  63,  67,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         85,  86,  88,  89,  95,  97, 113], device='cud

<ipython-input-18-1d96a1356a99>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test3D=torch.load(os.path.join('DatasetSplit','test'+str(i)+'.pth'))


Sampling probability : 0.7398
Time index : 0 out of 15
tensor([ 31,  39,  40,  41,  49,  53,  95,  99, 105], device='cuda:0')
Test / Subject : 9 / Timepoint : 0 / Loss : 0.18685559928417206 / AUROC : 0.9955555555555555 / F1 : 0.9955357142857143 / AP : 0.9962182500030841

Time index : 1 out of 15
tensor([ 31,  36,  39,  40,  41,  43,  45,  47,  48,  49,  53,  58,  63,  67,
         94,  99, 102, 103, 105, 110, 113, 115], device='cuda:0')
Test / Subject : 9 / Timepoint : 1 / Loss : 0.1875666081905365 / AUROC : 0.9934786840568426 / F1 : 0.993435877339081 / AP : 0.9943894257953267

Time index : 2 out of 15
tensor([ 10,  36,  39,  40,  41,  43,  45,  47,  48,  49,  50,  51,  53,  58,
         62,  63,  67,  72,  75,  78,  80,  82,  83,  91,  93,  94,  99, 102,
        103, 105, 113], device='cuda:0')
Test / Subject : 9 / Timepoint : 2 / Loss : 0.1889793425798416 / AUROC : 0.9932196822936846 / F1 : 0.9931733957480008 / AP : 0.9942206006923048

Time index : 3 out of 15
tensor([  1,   2,   7, 

In [ ]:
# Calculating the recorded frequencies of all brain regions

connAmyg=torch.load('connDictTestAmygL.pth')
connAmyg = dict(sorted(connAmyg.items(), key=lambda item: item[1], reverse=True))
print(connAmyg)

values=list(connAmyg.values())
values.remove(max(values))
print(torch.quantile(torch.tensor(list(values), dtype=float), 0.75))

connIns=torch.load('connDictTestInsL.pth')
connIns = dict(sorted(connIns.items(), key=lambda item: item[1], reverse=True))
print(connIns)

values=list(connIns.values())
values.remove(max(values))
print(torch.quantile(torch.tensor(list(values), dtype=float), 0.75))

<ipython-input-4-9a98ebdfee23>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  connAmyg=torch.load('connDictTestAmygL.pth')


{'Amygdala_L': 147, 'Precuneus_R': 76, 'Rolandic_Oper_R': 69, 'Cuneus_R': 66, 'Parietal_Sup_R': 65, 'Precentral_R': 64, 'Temporal_Sup_R': 64, 'Occipital_Mid_R': 63, 'Frontal_Med_Orb_R': 62, 'Amygdala_R': 62, 'Occipital_Sup_L': 61, 'Temporal_Inf_R': 61, 'Cerebelum_3_L': 61, 'Occipital_Sup_R': 60, 'Fusiform_L': 59, 'Frontal_Inf_Oper_R': 59, 'Hippocampus_L': 58, 'Temporal_Mid_R': 58, 'Calcarine_L': 57, 'Parietal_Inf_L': 57, 'Putamen_L': 56, 'Caudate_R': 56, 'Cerebelum_4_5_R': 55, 'Precuneus_L': 55, 'Supp_Motor_Area_R': 54, 'Cingulum_Mid_R': 54, 'Frontal_Inf_Tri_R': 54, 'Olfactory_L': 54, 'Insula_L': 54, 'Calcarine_R': 53, 'Occipital_Inf_R': 53, 'Thalamus_L': 53, 'Parietal_Sup_L': 52, 'Rectus_L': 52, 'ParaHippocampal_L': 52, 'Fusiform_R': 51, 'Lingual_R': 51, 'Postcentral_R': 51, 'Temporal_Inf_L': 51, 'Postcentral_L': 51, 'Vermis_6': 50, 'Cuneus_L': 50, 'Frontal_Sup_Medial_R': 50, 'Frontal_Inf_Orb_L': 49, 'Temporal_Pole_Sup_L': 49, 'Cerebelum_4_5_L': 49, 'Occipital_Inf_L': 49, 'Cerebelum_6

<ipython-input-4-9a98ebdfee23>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  connIns=torch.load('connDictTestInsL.pth')


{'Insula_L': 144, 'Temporal_Sup_L': 112, 'Rolandic_Oper_L': 106, 'Insula_R': 104, 'Supp_Motor_Area_L': 96, 'Frontal_Inf_Oper_L': 95, 'Postcentral_L': 95, 'Supp_Motor_Area_R': 94, 'Temporal_Pole_Sup_L': 94, 'Rolandic_Oper_R': 92, 'Parietal_Inf_L': 92, 'SupraMarginal_L': 92, 'Frontal_Inf_Tri_L': 91, 'Cingulum_Mid_L': 88, 'SupraMarginal_R': 87, 'Frontal_Mid_L': 86, 'Frontal_Sup_L': 85, 'Parietal_Sup_L': 82, 'Temporal_Sup_R': 81, 'Frontal_Inf_Tri_R': 81, 'Postcentral_R': 80, 'Frontal_Mid_R': 77, 'Cingulum_Ant_L': 74, 'Precentral_R': 73, 'Heschl_L': 73, 'Lingual_R': 71, 'Hippocampus_L': 70, 'Cingulum_Mid_R': 70, 'Occipital_Sup_L': 69, 'Putamen_L': 69, 'Frontal_Mid_Orb_L': 68, 'Cuneus_R': 68, 'Calcarine_L': 68, 'Frontal_Inf_Orb_L': 67, 'ParaHippocampal_L': 67, 'Occipital_Mid_L': 67, 'Putamen_R': 66, 'Frontal_Inf_Oper_R': 66, 'Parietal_Sup_R': 63, 'Parietal_Inf_R': 63, 'Temporal_Inf_R': 62, 'Fusiform_L': 61, 'Thalamus_L': 61, 'Occipital_Sup_R': 61, 'Fusiform_R': 60, 'Frontal_Med_Orb_L': 60, '

The two cells below is for testing the model trained with the 80th Quantile Dataset.

In [ ]:
# 80th Quantile Dataset Testing

sampleStrategy='uniform'
negSampleStrategy='inductive'
N=116 # Number of ROIs
numEdges=math.ceil(116*116*0.4)
nodeDim=N # One-Hot encoding as a node embedding
edgeDim=4 # Arbitrarily set
timeDim=100 # Half of the value mentioned in DyGFormer - HOT
channelDim=50 # Half of the value mentioned in DyGFormer - HOT
latentDim=N # Positive node embedding and negative node embedding concatenated across dim=1
numFilter=4 # Following Triadic Decoder paper
B=10000
prob=0.7398
nodeFeat=torch.cat((torch.eye(N),torch.zeros((1,N))),dim=0).numpy()
edgeFeat=torch.zeros((numEdges+1,edgeDim)).numpy()

checkpoint=torch.load('checkpoint9.pth')
wandb.init(
    project="EmotionConnectivity - 80thQuantileTest",
    config={
        "Mode" : "Test",
        "initialization" : "kaiming_normal",
        "sampleStrategy" : sampleStrategy,
        "negSampleStrategy" : negSampleStrategy,
        "B": B,
        "prob": prob,
        "Dropout" : 0.1,
    }
)


encoder=HOT(nodeFeat, edgeFeat, None, timeDim, channelDim, patch_size=8, num_state_vectors=32, num_layers=2, num2hop=0, dropout=0.1, max_input_sequence_length=4096, device='cuda')
encoder.apply(init_kaiming_normal)
decoder=TriadicDecoder(latentDim, numFilter, device='cuda')
decoder.apply(init_kaiming_normal)
model=nn.Sequential(encoder, decoder).to('cuda')
lossFunc=nn.BCELoss().to('cuda')
optimizerE=torch.optim.Adam(model.parameters(), lr=checkpoint['lr'])
data=datasets.fetch_atlas_aal()
connDictAmygL={}
connDictInsL={}


if os.path.isfile('checkpoint9.pth'):
  checkpoint=torch.load('checkpoint9.pth', map_location='cuda')
for i in range(0, 10):
  if os.path.isfile('checkpoint9.pth'):
    checkpoint=torch.load('checkpoint9.pth', map_location='cuda')
  model.eval()
  if checkpoint['encoder'] is not None: model[0].load_state_dict(checkpoint['encoder'])
  if checkpoint['decoder'] is not None: model[1].load_state_dict(checkpoint['decoder'])
  print("\nSubject index : "+str(i)+" out of 10")

  test403D=torch.load(os.path.join('DatasetSplit','test80'+str(i)+'.pth'))
  test403D=test403D.reshape(-1,3).detach().to('cpu').numpy()
  test403D=test403D.reshape(15, -1, 3)

  print("Sampling probability : "+str(prob))

  lossSubject=torch.zeros(1).to('cuda')
  lossSubject.requires_grad_()
  F1Subject=0
  APSubject=0
  AUROCSubject=0
  for t in range(0, 15):
    print("Time index : "+str(t)+" out of 15")

    timeBatch=test403D[t]
    srcNodes=timeBatch[:,0]
    dstNodes=timeBatch[:,1]
    timepoints=timeBatch[:,2]

    adjMat=torch.zeros((N,N)).to('cuda')
    for row in timeBatch:
      adjMat[int(row[0]), int(row[1])]=1

    nodePosEmbedding=torch.zeros((N,N)).to('cuda')
    nodeNegEmbedding=torch.zeros((N,N)).to('cuda')
    nodePosTimes=torch.zeros(N).to('cuda')
    nodeNegTimes=torch.zeros(N).to('cuda')
    nodeEmbedding=torch.zeros((N,N)).to('cuda')

    # Sample neighbor edges among the dynamic graphs of a single subject
    test_neighbor_sampler = get_neighbor_sampler(data=timeBatch, sample_neighbor_strategy=sampleStrategy, seed=4)
    # Sample negative edges among the dynamic graphs of a single subject
    test_neg_edge_sampler = NegativeEdgeSampler(src_node_ids=srcNodes, dst_node_ids=dstNodes, interact_times=timepoints, last_observed_time=np.min(timepoints), negative_sample_strategy=negSampleStrategy, seed=5)
    model[0].set_neighbor_sampler(test_neighbor_sampler)

    subject_src_node_embeddings, subject_dst_node_embeddings = \
      torch.utils.checkpoint.checkpoint(model[0].compute_src_dst_node_temporal_embeddings, srcNodes, dstNodes, timepoints, use_reentrant=False)
    negSrcNodes, negDstNodes = test_neg_edge_sampler.sample(len(srcNodes), srcNodes, dstNodes, np.min(timepoints), np.max(timepoints))
    subject_neg_src_node_embeddings, subject_neg_dst_node_embeddings = \
      torch.utils.checkpoint.checkpoint(model[0].compute_src_dst_node_temporal_embeddings, negSrcNodes, negDstNodes, timepoints, use_reentrant=False)

    for tb, row in enumerate(zip(srcNodes, dstNodes)):
      nodePosEmbedding[row[0]]=nodePosEmbedding[row[0]]+subject_src_node_embeddings[tb]
      nodePosTimes[row[0]]+=1
      nodePosEmbedding[row[1]]=nodePosEmbedding[row[1]]+subject_dst_node_embeddings[tb]
      nodePosTimes[row[1]]+=1
    for tb, row in enumerate(zip(negSrcNodes, negDstNodes)):
      nodeNegEmbedding[row[0]]=nodeNegEmbedding[row[0]]+subject_neg_src_node_embeddings[tb]
      nodeNegTimes[row[0]]+=1
      nodeNegEmbedding[row[1]]=nodeNegEmbedding[row[1]]+subject_neg_dst_node_embeddings[tb]
      nodeNegTimes[row[1]]+=1

    for node in range(N):
      if nodePosTimes[node]>0:
        nodePosEmbedding[node]=nodePosEmbedding[node]/nodePosTimes[node]
      if nodeNegTimes[node]>0:
        nodeNegEmbedding[node]=nodeNegEmbedding[node]/nodeNegTimes[node]


    nodePosEmbedding=(nodePosEmbedding-torch.mean(nodePosEmbedding, dim=0, keepdims=True))/(torch.std(nodePosEmbedding, dim=0, keepdims=True)+1e-10)
    nodeNegEmbedding=(nodeNegEmbedding-torch.mean(nodeNegEmbedding, dim=0, keepdims=True))/(torch.std(nodeNegEmbedding, dim=0, keepdims=True)+1e-10)

    graph2D=np.concatenate((np.expand_dims(srcNodes, axis=1), np.expand_dims(dstNodes, axis=1)), axis=1)
    graph2D=torch.tensor(graph2D).to('cuda')
    graph2DN=np.concatenate((np.expand_dims(negSrcNodes, axis=1), np.expand_dims(negDstNodes, axis=1)), axis=1)
    graph2DN=torch.tensor(graph2DN).to('cuda')
    triads=triadSample(graph2D, B, prob, device='cuda')

    mSquare=torch.zeros((N,N)).to('cuda')
    eSquare=torch.zeros((N,N)).to('cuda')

    for j, triad in enumerate(triads):
      triad0=triad[0].unsqueeze(0).unsqueeze(0)
      triad1=triad[1].unsqueeze(0).unsqueeze(0)
      triad2=triad[2].unsqueeze(0).unsqueeze(0)

      if mSquare[triad[0], triad[1]]==0:
        pair1=torch.cat((triad0, triad1), dim=1).to('cuda')
        M1=calculateM(triads, pair1)
        mSquare[triad[0], triad[1]]=M1
      if mSquare[triad[0], triad[2]]==0:
        pair2=torch.cat((triad0, triad2), dim=1).to('cuda')
        M2=calculateM(triads, pair2)
        mSquare[triad[0], triad[2]]=M2
      if mSquare[triad[1], triad[2]]==0:
        pair3=torch.cat((triad1, triad2), dim=1).to('cuda')
        M3=calculateM(triads, pair3)
        mSquare[triad[1], triad[2]]=M3

    for j, triad in enumerate(triads):
      triad0=triad[0].unsqueeze(0).unsqueeze(0)
      triad1=triad[1].unsqueeze(0).unsqueeze(0)
      triad2=triad[2].unsqueeze(0).unsqueeze(0)
      pair1=torch.cat((triad0, triad1), dim=1).squeeze(0).to('cuda')
      pair2=torch.cat((triad0, triad2), dim=1).squeeze(0).to('cuda')
      pair3=torch.cat((triad1, triad2), dim=1).squeeze(0).to('cuda')

      emb1=torch.zeros((1,N)).to('cuda')
      emb2=torch.zeros((1,N)).to('cuda')
      emb3=torch.zeros((1,N)).to('cuda')

      if pairIn(pair1,graph2D[:,0:2]):
        emb1=nodePosEmbedding[triad[0]]
      if pairIn(pair1,graph2DN):
        emb1=nodeNegEmbedding[triad[0]]
      if pairIn(pair2,graph2D[:,0:2]):
        emb2=nodePosEmbedding[triad[1]]
      if pairIn(pair2,graph2DN):
        emb2=nodeNegEmbedding[triad[1]]
      if pairIn(pair3,graph2D[:,0:2]):
        emb3=nodePosEmbedding[triad[2]]
      if pairIn(pair3,graph2DN):
        emb3=nodeNegEmbedding[triad[2]]

      eTriplet=model[1].forward(emb1, emb2, emb3)
      if pairIn(pair1,graph2D[:,0:2]):
        eSquare[triad[0], triad[1]]+=eTriplet[0]
      if pairIn(pair1,graph2DN):
        eSquare[triad[0], triad[1]]-=eTriplet[0]
      if pairIn(pair2,graph2D[:,0:2]):
        eSquare[triad[0], triad[2]]+=eTriplet[1]
      if pairIn(pair2,graph2DN):
        eSquare[triad[0], triad[2]]-=eTriplet[1]
      if pairIn(pair3,graph2D[:,0:2]):
        eSquare[triad[1], triad[2]]+=eTriplet[2]
      if pairIn(pair3,graph2DN):
        eSquare[triad[1], triad[2]]-=eTriplet[2]
    eSquare.clamp_(min=0.0)

    nonzeros=((mSquare>0).nonzero())
    adjRecon=[]
    adjPart=[]
    adjFull=torch.zeros((N,N)).to('cuda')
    for nz in nonzeros:
      nzPair=torch.tensor([int(nz[0]), int(nz[1])]).to('cuda')
      em1=eSquare[int(nz[0]), int(nz[1])]/mSquare[int(nz[0]), int(nz[1])]
      if pairIn(nzPair,graph2D[:,0:2]):
        adjRecon.append(em1)
        adjPart.append(torch.tensor([1.0]))
      if pairIn(nzPair, graph2DN):
        adjRecon.append(em1)
        adjPart.append(torch.tensor([0.0]))
      if em1>0.5:
        adjFull[int(nz[0]), int(nz[1])]=1
      else:
        adjFull[int(nz[0]), int(nz[1])]=0
    adjRecon=torch.stack(adjRecon).to('cuda')
    adjPart=torch.stack(adjPart).squeeze(1).to('cuda')

    loss=lossFunc(input=adjRecon, target=adjPart)
    lossSubject=lossSubject+loss

    adjReconC=adjRecon.clone().detach().cpu().numpy()
    adjPartC=adjPart.clone().detach().cpu().numpy()


    adjReconC[adjReconC>0.5]=1
    adjReconC[adjReconC<=0.5]=0
    F1=f1_score(adjPartC, adjReconC)
    AUROC=roc_auc_score(adjPartC, adjReconC)
    AP=average_precision_score(adjPartC, adjReconC)
    F1Subject+=F1
    APSubject+=AP
    AUROCSubject+=AUROC

    print("Test / Subject : "+str(i)+" / Timepoint : "+str(t) +" / Loss : "+str(loss.item())+" / AUROC : "+str(AUROC)+" / F1 : "+str(F1)+" / AP : "+str(AP)+"\n")

    AmygLConn=torch.nonzero(adjFull[40])[:,0]
    for elem in AmygLConn:
      if data.labels[elem] not in connDictAmygL.keys():
        connDictAmygL[data.labels[elem]]=1
      else:
        connDictAmygL[data.labels[elem]]+=1

    InsLConn=torch.nonzero(adjFull[28])[:,0]
    for elem in InsLConn:
      if data.labels[elem] not in connDictInsL.keys():
        connDictInsL[data.labels[elem]]=1
      else:
        connDictInsL[data.labels[elem]]+=1

  wandb.log({"TLoss": lossSubject.item()/15, "T_AP": APSubject/15, "T_F1": F1Subject/15, "T_AUROC":AUROCSubject/15})
  torch.cuda.empty_cache()

torch.save(connDictAmygL, 'connDictTestAmygL40.pth')
torch.save(connDictInsL, 'connDictTestInsL40.pth')

<ipython-input-11-54ea28611265>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load('checkpoint9.pth')


<ipython-input-11-54ea28611265>:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load('checkpoint9.pth', map_location='cuda')
<ipython-input-11-54ea2861126


Subject index : 0 out of 10
Sampling probability : 0.7398
Time index : 0 out of 15


<ipython-input-11-54ea28611265>:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test403D=torch.load(os.path.join('DatasetSplit','test40'+str(i)+'.pth'))


tensor(0.3846, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 1, 40, 95], device='cuda:0')
Test / Subject : 0 / Timepoint : 0 / Loss : 0.2437719851732254 / AUROC : 0.9509726861858911 / F1 : 0.9484450873024073 / AP : 0.9582595581295974

Time index : 1 out of 15
tensor(0.3864, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 0 / Timepoint : 1 / Loss : 0.24172593653202057 / AUROC : 0.9549911747401452 / F1 : 0.9528699045076496 / AP : 0.9617882185547824

Time index : 2 out of 15
tensor(0.3859, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 0 / Timepoint : 2 / Loss : 0.2384541630744934 / AUROC : 0.957004160887656 / F1 : 0.9550724637681159 / AP : 0.9632705238184449

Time index : 3 out of 15
tensor(0.3857, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 0 / Timepoint : 3 / Loss : 0.23961371183395386 / AUROC : 0.9555643721483833 / F1 : 0.95349802

...

Sampling probability : 0.7398
Time index : 0 out of 15
tensor(0.3796, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 7 / Timepoint : 0 / Loss : 0.24345709383487701 / AUROC : 0.9525348414461725 / F1 : 0.9501696352841391 / AP : 0.9590926713980921

Time index : 1 out of 15
tensor(0.3820, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 7 / Timepoint : 1 / Loss : 0.242991641163826 / AUROC : 0.9527308838133068 / F1 : 0.9503856577027309 / AP : 0.9594912795448883

Time index : 2 out of 15
tensor(0.3776, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0, 40], device='cuda:0')
Test / Subject : 7 / Timepoint : 2 / Loss : 0.24260054528713226 / AUROC : 0.9529182879377431 / F1 : 0.9505920783993467 / AP : 0.9591816657024455

Time index : 3 out of 15
tensor(0.3823, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 1, 40], device='cuda:0')
Test / Subject : 7 / Timepoint : 3 / Loss : 0.2417932450771331

<ipython-input-11-54ea28611265>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load('checkpoint9.pth', map_location='cuda')
<ipython-input-11-54ea2861126

Sampling probability : 0.7398
Time index : 0 out of 15
tensor(0.3846, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 8 / Timepoint : 0 / Loss : 0.24377255141735077 / AUROC : 0.9518648701425503 / F1 : 0.9494307108421377 / AP : 0.959023594031175

Time index : 1 out of 15
tensor(0.3814, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([0, 1], device='cuda:0')
Test / Subject : 8 / Timepoint : 1 / Loss : 0.24033187329769135 / AUROC : 0.9544628432956381 / F1 : 0.9522902782185549 / AP : 0.9607588092208591

Time index : 2 out of 15
tensor(0.3927, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 8 / Timepoint : 2 / Loss : 0.249963641166687 / AUROC : 0.9513386750048857 / F1 : 0.9488496302382908 / AP : 0.9598070964821815

Time index : 3 out of 15
tensor(0.3954, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 1, 41], device='cuda:0')
Test / Subject : 8 / Timepoint : 3 / Loss : 0.24873854219913483 /

<ipython-input-11-54ea28611265>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint=torch.load('checkpoint9.pth', map_location='cuda')
<ipython-input-11-54ea2861126

Sampling probability : 0.7398
Time index : 0 out of 15
tensor(0.3868, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 1, 38, 40], device='cuda:0')
Test / Subject : 9 / Timepoint : 0 / Loss : 0.24745668470859528 / AUROC : 0.9503567181926278 / F1 : 0.9477635283077885 / AP : 0.9581478052950344

Time index : 1 out of 15
tensor(0.3871, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 9 / Timepoint : 1 / Loss : 0.24714255332946777 / AUROC : 0.9498818432453722 / F1 : 0.9472374831553851 / AP : 0.957775995585798

Time index : 2 out of 15
tensor(0.3911, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 9 / Timepoint : 2 / Loss : 0.2513614594936371 / AUROC : 0.9482254290171607 / F1 : 0.9453984575835476 / AP : 0.9571088429555002

Time index : 3 out of 15
tensor(0.3884, device='cuda:0', grad_fn=<MeanBackward0>)
tensor([ 0,  1, 40], device='cuda:0')
Test / Subject : 9 / Timepoint : 3 / Loss : 0.24874988

In [ ]:
# Calculating the recorded frequencies of all brain regions

connAmyg=torch.load('connDictTestAmygL40.pth')
connAmyg = dict(sorted(connAmyg.items(), key=lambda item: item[1], reverse=True))
print(connAmyg)

values=list(connAmyg.values())
values.remove(max(values))
print(torch.quantile(torch.tensor(list(values), dtype=float), 0.75))

connIns=torch.load('connDictTestInsL40.pth')
connIns = dict(sorted(connIns.items(), key=lambda item: item[1], reverse=True))
print(connIns)

values=list(connIns.values())
values.remove(max(values))
print(torch.quantile(torch.tensor(list(values), dtype=float), 0.75))

{'Amygdala_L': 121, 'Precentral_R': 118, 'Precentral_L': 107, 'Frontal_Sup_L': 22, 'Frontal_Sup_R': 5, 'Temporal_Pole_Sup_L': 3, 'Cerebelum_10_R': 3, 'Frontal_Mid_Orb_R': 2, 'Amygdala_R': 2, 'Olfactory_L': 2, 'Cerebelum_3_R': 1, 'Hippocampus_L': 1, 'Pallidum_L': 1, 'Caudate_R': 1, 'Frontal_Sup_Orb_L': 1, 'Pallidum_R': 1, 'Temporal_Pole_Mid_L': 1, 'Vermis_6': 1, 'ParaHippocampal_L': 1, 'Thalamus_L': 1, 'Cerebelum_3_L': 1}
tensor(3., dtype=torch.float64)
{'Insula_L': 139, 'Rolandic_Oper_L': 104, 'Insula_R': 99, 'Frontal_Mid_L': 96, 'Cingulum_Mid_R': 95, 'Supp_Motor_Area_L': 94, 'Postcentral_L': 94, 'Parietal_Inf_L': 94, 'Temporal_Sup_L': 94, 'Frontal_Inf_Tri_R': 88, 'Supp_Motor_Area_R': 88, 'Frontal_Inf_Oper_L': 85, 'Frontal_Inf_Tri_L': 85, 'Cingulum_Mid_L': 85, 'Calcarine_R': 85, 'SupraMarginal_L': 82, 'Frontal_Mid_R': 82, 'SupraMarginal_R': 81, 'Rolandic_Oper_R': 79, 'Parietal_Sup_L': 79, 'Frontal_Inf_Orb_L': 77, 'Cingulum_Ant_L': 77, 'Lingual_L': 75, 'Temporal_Sup_R': 74, 'Frontal_Sup

<ipython-input-12-c18a1a48f76c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  connAmyg=torch.load('connDictTestAmygL40.pth')
<ipython-input-12-c18a1a48f76c>:9: FutureWarn